# Census of Immune Cells: Single-Cell Workflow with CoGAPS

# Authors
Tom Sherman - <tomsherman159@gmail.com>  
Ted Liefeld - <jliefeld@cloud.ucsd.edu>  
Emily Davis - <edavis71@jhu.edu>  
Genevieve Stein-O'Brien - <gsteinobrien@jhmi.edu>  
Michael Reich - <mmreich@cloud.ucsd.edu>  
Elana Fertig - <ejfertig@jhmi.edu>  

# Table of Contents
<a href=#Introduction>Introduction</a>  
<a href=#Login>Login to GenePattern</a>  
<a href=#Prerequisites>Prerequisites</a>  
<a href=#Download-Raw-Data>Download Raw Data</a>  
<a href=#Add-Preliminary-Annotations>Add Preliminary Annotations</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Experimental-Conditions>Experimental Conditions</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Count-Information>Count Information</a>  
<a href=#Filter-Low-Quality-Samples>Filter Low Quality Samples</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Visualize-Overall-Distribution>Visualize Overall Distribution</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Cell-Filter----Count-Threshold>Cell Filter - Count Threshold</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Cell-Filter---Gene-Threshold>Cell Filter - Gene Threshold</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Remaining-Cells>Remaining Cells</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Use-the-ScanpyUtilities-Module-to-Filter>Use the ScanpyUtilities Module to Filter</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Subset-Data-for-Further-Analysis>Subset Data for Further Analysis</a>  
<a href=#Cell-Type-Identification>Cell Type Identification</a>  
<a href=#Final-Pre-Processing-Steps>Final Pre-Processing Steps</a>  
<a href=#Visualization>Visualization</a>  
<a href=#CoGAPS-Analysis>CoGAPS Analysis</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Setting-up-a-CoGAPS-Run>Setting up a CoGAPS Run</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Processing-the-CoGAPS-Result>Processing the CoGAPS Result</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Visualizing-Patterns>Visualizing Patterns</a>  
&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Analyzing-Patterns>Analyzing Patterns</a>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Known-Phenotypes>Known Phenotypes</a>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Cell-Type-Identification-with-CoGAPS>Cell Type Identification with CoGAPS</a>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href=#Exploring-the-Patterns>Exploring the Patterns</a>  
<a href=#References>References</a>  

# Introduction
<a href=#Table-of-Contents>back to top</a>  

It is not uncommon to find single-cell data sets with hundreds of thousands, or even millions, of cells. The size of these data sets presents a challenge for most analysis pipelines, often requiring an external server to run. Working in this way can provide an unfamiliar environment for many researchers and can hinder their ability to focus on the actual analysis. In this notebook we walk through a common preprocessing workflow on a RNA-seq data set with 250,000 cells. Furthermore, we run a computationally intensive matrix factorization algorithm, **CoGAPS** [2,6], completely from the interface provided in the notebook. We demonstrate how the GenePattern notebook environment combines the user friendliness of a Jupyter notebook with the compute power of a large cluster.

The data set used in this analysis is one of the Human Cell Atlas [preview datasets](https://preview.data.humancellatlas.org/) [1]. We will be looking at the subset of umbilical cord blood cells from the Census of Immune Cells dataset. There are roughly 250,000 cells in the cord blood subset, taken from 8 donors. For each donor there were 8 independent 10x channels prepared, with the top 6000 cells (according to **CellRanger**) from each channel included in the data set. For more information about this data and how it was generated, refer to the documention at the preview link and the press release [here](https://www.broadinstitute.org/news/researchers-post-genetic-profiles-half-million-human-immune-cells-human-cell-atlas-online).

In this workflow we will roughly follow a standard preprocessing pipeline outlined [here](https://github.com/theislab/single-cell-tutorial/) with one notable exception. Rather than "manually" labeling cell types, we will use the R package **garnett** [4] to automatically label the cell types based on a list of provided marker genes. Most of the preprocessing steps are done with the python package **scanpy** [8]. This will be the default package used for visualization and recording all the information learned from our analysis. Once the data has been preprocessed we will apply a matrix factorization algorithm, CoGAPS, to further analyze the structure present in this data set as a function of learned cell types and individuals.

Both **scanpy** and **CoGAPS** will require compute power that is not available in the notebook locally. This is mainly due to the large memory requirements of analyzing a data set of this size. To handle this, we will use GenePattern modules which wrap up standard Python/R functionality and allow us to submit jobs to a server through the notebook environment. When our job finishes we receive a notification and go about working interactively in the notebook. We will make use of two modules in particular, *ScanpyUtilities* and *CoGAPS*. The *CoGAPS* module provides a wrapper around the **CoGAPS** R package and the *ScanpyUtilities* module wraps several commonly used functions when preprocessing data with **scanpy**. The *ScanpyUtilities* module also wraps a normalization pipeline with the R package **scran** [3] and a cell type identification pipeline with the R package **garnett**. These pipelines still work with the native file format for the **scanpy** package, so they will integrate seemlessly with the full analysis pipeline.

<div class="alert alert-info">
Follow instructions that look like this in order to reproduce the original analysis done in this notebook.
</div>

# Login
<a href=#Table-of-Contents>back to top</a>  

Make sure to login here before proceeding any farther in the notebook. The module output won't appear until you are logged in.

In [1]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

# Prerequisites
<a href=#Table-of-Contents>back to top</a>  

We first have to make sure all dependences are installed and all package versions are correct. If you run the following cells, these checks will be performed automatically.

In [8]:
## this notebook requires Python 3.7
import sys
def CheckPythonVersion():
    assert sys.version_info >= (3, 7)
    print("Notebook is compatible with Python version")

genepattern.GPUIBuilder(CheckPythonVersion, 
                        description="Check that the current Python version is sufficient to run the notebook.",
                        parameters={"output_var": {"hide": True}})

UIBuilder(description='Check that the current Python version is sufficient to run the notebook.', function_imp…

In [9]:
## this cell loads all the neccesary python packages and checks to see that the versions are sufficient
## it also initializes the R/python environment needed for using %%R to run R code

import scanpy.api as sc
import numpy as np
import pandas as pd
import anndata
import h5py
import random
from scipy import sparse, io
import scipy.stats as ss

import urllib.request
import os
import sys

import matplotlib
import seaborn as sb
import matplotlib.pyplot as plt

import gp
import genepattern

import tzlocal
import reprlib
import re

# function that checks that the pacakge version is greater or equal to the target
def checkVersion(pkg, targetVersion):
    pkgVersion = [int(i) for i in pkg.__version__.split('.')]
    while len(targetVersion) < len(pkgVersion):
        targetVersion += (0,)
    while len(pkgVersion) < len(targetVersion):
        pkgVersion += (0,)
    return tuple(pkgVersion) >= targetVersion

def SetupPythonEnvironment():
    assert(checkVersion(sc, (1,4)))
    print("scanpy package is up to date")
    assert(checkVersion(anndata, (0,6,18)))
    print("anndata package is up to date")
    print("python setup successful")

genepattern.GPUIBuilder(SetupPythonEnvironment, 
                        description="Loads all neccesary Python packages and makes sure versions are up to date.",
                        parameters={"output_var": {"hide": True}})

UIBuilder(description='Loads all neccesary Python packages and makes sure versions are up to date.', function_…

When you run the next cell for the first time, it may take a while (~30 minutes) since there are several R packages that need be installed locally before you can use this notebook. Don't be alarmed if this next cell produces a lot of warnings. Most of them are just standard output messages from the installation process that are mistakenly interpreted as warnings.

In [10]:
def SetupREnvironment():
    ## this sets up the directory for any installed R packages
    !mkdir -p ~/Rpackages
    !echo R_LIBS=~/Rpackages > ~/.Renviron
    
    # allows R code to be run from python
    %load_ext rpy2.ipython
    
    # function to check if package is installed without loading it
    %R packageInstalled <- function(pkgName) return(pkgName %in% rownames(installed.packages()))

    # check for installation tools
    %R if (!packageInstalled("BiocManager")) install.packages("BiocManager", lib="~/Rpackages")
    %R if (!packageInstalled("Matrix")) BiocManager::install("Matrix", ask=FALSE, update=FALSE, lib="~/Rpackages")
    %R if (!packageInstalled("rhdf5")) BiocManager::install("rhdf5", ask=FALSE, update=FALSE, lib="~/Rpackages")
    %R if (!packageInstalled("dunn.test")) BiocManager::install("dunn.test", ask=FALSE, update=FALSE, lib="~/Rpackages")

    # install most recent version of CoGAPS
    %R if (!packageInstalled("CoGAPS")) BiocManager::install("CoGAPS", ask=FALSE, update=FALSE, lib="~/Rpackages")
    %R print(paste("CoGAPS version:", packageVersion("CoGAPS")))
    
    %R print(ifelse(all(packageInstalled(c("BiocManager", "Matrix", "rhdf5", "dunn.test", "CoGAPS"))), "R setup successful", "R setup failed"))

genepattern.GPUIBuilder(SetupREnvironment, 
                        description="Installs and loads all neccesary R packages and makes sure versions are up to date.",
                        parameters={"output_var": {"hide": True}})

UIBuilder(description='Installs and loads all neccesary R packages and makes sure versions are up to date.', f…

This next cell will load all the functions that support the UI cells you see throughout the notebook. 

In [11]:
#### Running this cell actually does nothing, the functions are loaded when the UI cell is created.
#### But if the user is able to hit the "run" button then the UI cell must have already been created
#### so we know the functions are loaded.

# global variable used to define a Which_Axis parameter, used to distinguish between samples and features
which_axis_param = {"default": "samples", "type": "choice", "choices": {"samples": "samples", "features": "features"}}    
vis_coordinates_param = {"default": "tsne", "type": "choice", "choices": {"T-SNE": "TSNE", "UMAP": "UMAP"}}
bool_param = {"default": "FALSE", "type": "choice", "choices": {"FALSE": False, "TRUE": True}}

#### Helper Functions ####

def printAndFlush(str):
    print(str)
    sys.stdout.flush()
    
#### CreateDirectory ####

CreateDirectory_description = "Create a new directory."
CreateDirectory_parameters = {"output_var": {"hide": True}}

def CreateDirectory(Directory_Name):
    !test -d {Directory_Name} && echo "Directory already exists!" || echo "Creating directory"
    !mkdir -p {Directory_Name}

#### DownloadFile ####

DownloadFile_description = "Download a file from the given URL."
DownloadFile_parameters = {"output_var": {"hide": True}}

def DownloadFile(URL, File_Name):
    if not os.path.isfile(File_Name):
        print("downloading file...")
        urllib.request.urlretrieve(URL, File_Name)
        print("done")
    else:
        print("File already downloaded")
        
#### DownloadJobResultFile ####

# Notes: write a job result file to a local directory, this function
# reads the file in chunks so it is memory efficient

DownloadJobResultFile_description = "Download file from a GenePattern module job result."
DownloadJobResultFile_parameters = {"Job_File_Url": {"type": "file", "kinds": ["h5ad", "rds"]}, "output_var": {"hide": True}}   
        
def DownloadJobResultFile(Job_File_Url, File_Name):
    if os.path.isfile(File_Name):
        print("file already exists!")
        return

    # extract job number and file name from url
    job_num = Job_File_Url.split("/")[-2]
    remote_file_name = Job_File_Url.split("/")[-1]
    
    # get the job based on the job number passed as the second argument
    job = gp.GPJob(genepattern.get_session(0), job_num)

    # fetch a specific file from the job
    remote_file = job.get_file(remote_file_name)
    
    printAndFlush("downloading file...")
    response = remote_file.open()
    CHUNK = 16 * 1024
    with open(File_Name, 'wb') as f:
        while True:
            chunk = response.read(CHUNK)
            if not chunk:
                break
            f.write(chunk)
    printAndFlush("done!")

#### InspectHDF5 ####

InspectHDF5_description = "List the contents of an HDF5 file."
InspectHDF5_parameters = {"output_var": {"hide": True}}    
    
def InspectHDF5(File_Name):
    %R -i File_Name -o h5_dir h5_dir <- rhdf5::h5ls(File_Name)
    return h5_dir

#### View RandomSubset ####

ViewRandomSubset_description =  "View a random subset of the AnnData object."
ViewRandomSubset_parameters = {"Which_Axis": which_axis_param, "output_var": {"hide": True}}

def ViewRandomSubset(AnnData_File, Num_Values, Which_Axis):
    adata = sc.read(AnnData_File, backed='r+')
    df = adata.obs if Which_Axis == 'samples' else adata.var
    ndx = random.sample(range(len(df.index)), Num_Values)
    if df.empty:
        print("DataFrame is empty")
        print("Variable names:", df.index[ndx])
        return
    print(df.iloc[ndx,:])

#### SwapIndexAndColumn ####

SwapIndexAndColumn_description = "Create a new index for the data using an existing column."
SwapIndexAndColumn_parameters = {"Which_Axis": which_axis_param, "output_var": {"hide": True}}

def SwapIndexAndColumn(AnnData_File, Index_Column, New_Column_Name, Which_Axis):
    adata = sc.read(AnnData_File, backed='r+')
    if Which_Axis == 'samples':
        df = adata.obs
    else:
        df = adata.var
        
    if Index_Column not in df.keys():
        print("ERROR: can't find index column")
        return
    if New_Column_Name in df.keys():
        print("ERROR: new column name already exists")
        return
    df[New_Column_Name] = df.index
    df.index = pd.Index(df[Index_Column])
    df.index.name = 'index'
    df.drop(Index_Column, 1, inplace=True)
    adata.write(compression='gzip', compression_opts=1, force_dense=False)

#### View ####
    
View_description = "View contents of variable."
View_parameters = {"output_var": {"hide": True}}

def View(Variable_Name):
    print(reprlib.repr(Variable_Name))

#### RegexVariableNames ####

RegexVariableNames_description = "Parse AnnData column using regex pattern."
RegexVariableNames_parameters = {"Which_Axis": which_axis_param,
                                 "output_var": {"name": "variable name for regex output", "hide": False}}
    
def RegexVariableNames(AnnData_File, Pattern, Which_Axis):
    adata = sc.read(AnnData_File, backed='r+')
    if Which_Axis == 'samples':
        return [re.search(Pattern, x).group(0) for x in adata.obs_names]
    else:
        return [re.search(Pattern, x).group(0) for x in adata.var_names]
    
#### AddColumnToData ####

AddColumnToData_description = "Adds a column to an AnnData object."
AddColumnToData_parameters = {"Which_Axis": which_axis_param, "output_var": {"hide": True}}

def AddColumnToData(AnnData_File, Column_Name, Values, Which_Axis):
    adata = sc.read(AnnData_File, backed='r+')
    df = adata.obs if Which_Axis == 'samples' else adata.var
    if Column_Name in df.keys():
        print("Error: column already exists")
        return
    df[Column_Name] = Values
    adata.write(compression='gzip', compression_opts=1, force_dense=False)

#### UploadFileToGenePatternServer ####

UploadFileToGenePatternServer_description = "Upload the local file onto the GenePattern server."
UploadFileToGenePatternServer_parameters = {"output_var": {"name": "variable name for url", "hide": False}}

def UploadFileToGenePatternServer(Local_File, Remote_File_Name):
    uploaded = genepattern.get_session(0).upload_file(Remote_File_Name, Local_File)
    print(uploaded.get_url())
    return uploaded.get_url()

#### ViolinPlot ####

ViolinPlot_description = "Create a violin plot from an AnnData object stored in a file."
ViolinPlot_parameters = {"output_var": {"hide": True}}

def ViolinPlot(AnnData_File, Plot_Variable, Group_Variable):
    adata = sc.read(AnnData_File, backed='r+')
    print("Creating Plot...")
    sc.pl.violin(adata, keys=Plot_Variable, groupby=Group_Variable, log=True, size=2, cut=0)

#### Histogram ####

Histogram_description = "Create a histogram of an AnnData object."
Histogram_parameters = {"Which_Axis": which_axis_param,
                        "Lower_Bound": {"type": "number", "optional": True},
                        "Upper_Bound": {"type": "number", "optional": True},
                        "output_var": {"hide": True}}

def Histogram(AnnData_File, Plot_Variable, Which_Axis, Lower_Bound, Upper_Bound):
    adata = sc.read(AnnData_File, backed='r+')
    print("Creating histogram of", Plot_Variable, "for the", Which_Axis)
    plot_data = adata.obs if Which_Axis == 'samples' else adata.var
    ndx = pd.Series([True for x in range(len(plot_data.index))])
    ndx.index = plot_data.index
    if Lower_Bound != '':
        print("Using lower bound of:", Lower_Bound)
        ndx = plot_data[Plot_Variable] >= Lower_Bound
    if Upper_Bound != '':
        print("Using upper bound of:", Upper_Bound)
        ndx = ndx & (plot_data[Plot_Variable] <= Upper_Bound)
    sb.distplot(plot_data[Plot_Variable][ndx], kde=False, bins=60)
    
#### CellScatterPlot ####

CellScatterPlot_description = "Create a scatter plot of cell features."
CellScatterPlot_parameters = {"output_var": {"hide": True}}

def CellScatterPlot(AnnData_File, x_variable, y_variable):
    adata = sc.read(AnnData_File, backed='r+')
    print("Creating scatter plot of", x_variable, "vs", y_variable)
    sc.pl.scatter(adata, x_variable, y_variable)
    
#### CountCellsByGroup ####

CountCellsByGroup_description = "Get cell counts split by group."
CountCellsByGroup_parmeters = {"output_var": {"hide": True}}

def CountCellsByGroup(AnnData_File, Group_Variable):
    adata = sc.read(AnnData_File, backed='r+')
    for g in adata.obs[Group_Variable].unique():
        print(Group_Variable, g, "- number of cells:", sum(adata.obs[Group_Variable] == g))
    
#### SubsetData ####

SubsetData_description = "Subset a data set to only cells of a certain group."
SubsetData_parameters = {"output_var": {"hide": True}}

def SubsetData(AnnData_File, Output_File, Group_Variable, Group_Value):
    printAndFlush("reading data...")
    adata = sc.read(AnnData_File)
    printAndFlush("subsetting data...")
    adata = adata[adata.obs[Group_Variable] == Group_Value]
    printAndFlush("re-filtering lowly expressed genes...")
    sc.pp.filter_genes(adata, min_cells=20)
    printAndFlush("writing data...")
    adata.write(Output_File, compression='gzip', compression_opts=1, force_dense=False)
    del adata
    printAndFlush("done")

#### GenerateTextFile ####

GenerateTextFile_description = "Create a text file."
GenerateTextFile_parameters = {"output_var": {"hide": True}}

def GenerateTextFile(Text, File_Name):
    file = open(File_Name,"w")
    file.write(Text)
    file.close()
    print("File Created:", File_Name)
    !cat {File_Name}
    
#### TsnePlot ####

TsnePlot_description = "Create a t-sne plot of an AnnData object."
TsnePlot_parameters = {"Variable_Subset": {"optional": True}, "output_var": {"hide": True}}

def TsnePlot(AnnData_File, Variable, Variable_Subset):
    print("Creating Plot...")
    if Variable_Subset:
        adata = sc.read(AnnData_File)
        ndx = [i for i,e in enumerate(adata.obs[Variable]) if e in Variable_Subset]
        sc.pl.tsne(adata[ndx], color=Variable, use_raw=False)
    else:
        adata = sc.read(AnnData_File, backed='r+')
        sc.pl.tsne(adata, color=Variable, use_raw=False)    

#### UmapPlot ####

UmapPlot_description = "Create a UMAP plot of an AnnData object."
UmapPlot_parameters = {"Variable_Subset": {"optional": True}, "output_var": {"hide": True}}

def UmapPlot(AnnData_File, Variable, Variable_Subset):
    print("Creating Plot...")
    if Variable_Subset:
        adata = sc.read(AnnData_File)
        ndx = [i for i,e in enumerate(adata.obs[Variable]) if e in Variable_Subset]
        sc.pl.umap(adata[ndx], color=Variable, use_raw=False)
    else:
        adata = sc.read(AnnData_File, backed='r+')
        sc.pl.umap(adata, color=Variable, use_raw=False)
        
#### GetDataSize ####

GetDataSize_description = "Display number of features/samples in a data set."
GetDataSize_parameters = {"output_var": {"hide": True}}

def GetDataSize(AnnData_File):
    adata = sc.read(AnnData_File, backed='r+')
    print("Number of samples:", adata.shape[0])
    print("Number of features:", adata.shape[1])

#### GetDataSparsity ####

GetDataSparsity_description = "Display percentage of data that is zero."
GetDataSparsity_parameters = {"output_var": {"hide": True}}

def GetDataSparsity(AnnData_File):
    adata = sc.read(AnnData_File)
    sparsity = 1 - adata.X.getnnz() / (adata.shape[0] * adata.shape[1])
    print("Sparsity (percent zero) of", AnnData_File, ":", round(sparsity, 2))

#### CreateCogapsParameters ####

CreateCogapsParameters_description = "Create a CoGAPS parameters object and save it to a file."
CreateCogapsParameters_parameters = {"Single_Cell_Data": bool_param,
                                     "Enable_Sparse_Optimization": bool_param,
                                     "output_var": {"hide": True}}

def CreateCogapsParameters(Output_File_Name, Number_Of_Patterns, Number_Of_Iterations, Random_Seed, Single_Cell_Data, Enable_Sparse_Optimization, Feature_Names, Sample_Names, Distributed_Method, Number_Of_Sets):
    %R -i Output_File_Name -i Number_Of_Patterns -i Number_Of_Iterations -i Random_Seed -i Single_Cell_Data -i Enable_Sparse_Optimization -i Feature_Names -i Sample_Names -i Distributed_Method -i Number_Of_Sets 
    %R library(CoGAPS)
    %R params <- CogapsParams(nPatterns=Number_Of_Patterns, nIterations=Number_Of_Iterations, seed=Random_Seed, singleCell=Single_Cell_Data, sparseOptimization=Enable_Sparse_Optimization, distributed=Distributed_Method, geneNames=Feature_Names, sampleNames=Sample_Names)
    %R params <- setDistributedParams(params, nSets=Number_Of_Sets)
    %R print(params)
    %R saveRDS(params, file=paste(Output_File_Name, "rds", sep="."))
    %R print(paste("successfully created file:", paste(Output_File_Name, "rds", sep=".")))

#### ConvertToMtx ####

ConvertToMtx_description = "Convert AnnData file to mtx."
ConvertToMtx_parameters = {"output_var": {"hide": True}}

def ConvertToMtx(AnnData_File, Mtx_File):
    if os.path.isfile(Mtx_File):
        print("can't write: mtx file already exists!")
        return
    %R suppressMessages(library(Matrix))
    %R suppressMessages(library(rhdf5))
    %R h5disableFileLocking()
    %R -i AnnData_File
    %R raw_data <- as.numeric(h5read(AnnData_File, "/X/data"))
    %R indices <- as.integer(h5read(AnnData_File, "/X/indices"))
    %R indptr <- as.integer(h5read(AnnData_File, "/X/indptr"))
    %R counts <- sparseMatrix(i=indices+1, p=indptr, x=raw_data)
    printAndFlush("writing mtx file...")
    %R print(paste("rows:", nrow(counts), "columns:", ncol(counts)))
    %R -i Mtx_File writeMM(counts, file=Mtx_File)
    printAndFlush("done")
    
#### ViewCogapsResult ####

ViewCogapsResult_description = "View the properties of a Cogaps result file."
ViewCogapsResult_parameters = {"output_var": {"hide": True}}

def ViewCogapsResult(Result_File):
    %R library(CoGAPS)
    %R -i Result_File
    %R cogapsResult <- readRDS(Result_File)
    %R print(cogapsResult)
    %R print(getOriginalParameters(cogapsResult))
    %R print(paste("Created with CoGAPS version", getVersion(cogapsResult)))
    
#### MergeCogapsResultWithAnnData ####

MergeCogapsResultWithAnnData_description = "Adds the patterns from the Cogaps result to the AnnData file."
MergeCogapsResultWithAnnData_parameters = {"output_var": {"hide": True}}

def MergeCogapsResultWithAnnData(AnnData_File, Cogaps_Result_File, Output_File):
    if os.path.isfile(Output_File):
        print("can't write: output file already exists!")
        return
    
    %R library(CoGAPS)
    %R -i Cogaps_Result_File
    %R cogapsResult <- readRDS(Cogaps_Result_File)
    %R samplePatterns <- getSampleFactors(cogapsResult)
    %R sampleNames <- rownames(samplePatterns)
    %R featurePatterns <- getFeatureLoadings(cogapsResult)
    %R featureNames <- rownames(featurePatterns)
    %R pm <- patternMarkers(cogapsResult)
    %R patternRanks <- pm$PatternRanks
    %R -o samplePatterns -o sampleNames -o featurePatterns -o featureNames -o patternRanks
    
    adata = sc.read(AnnData_File)
    if not all(sampleNames == adata.obs.index):
        print("Error: mismatch in order of samples")
        return
    if not all(featureNames == adata.var.index):
        print("Error: mismatch in order of features")
        return
    nPatterns = samplePatterns.shape[1]
    adata.uns['nPatterns'] = nPatterns
    for i in range(nPatterns):
        adata.obs['cogaps.pattern.' + str(i+1)] = samplePatterns[:,i]
    for i in range(nPatterns):
        adata.var['cogaps.pattern.' + str(i+1)] = featurePatterns[:,i]
        adata.var['cogaps.pattern.rank.' + str(i+1)] = patternRanks[:,i].astype(int)
    adata.write(Output_File, compression='gzip', compression_opts=1, force_dense=False)

#### PlotCogapsPatterns ####

PlotCogapsPatterns_description = "Plots patterns overlaid on either a UMAP or TSNE plot of the original data."
PlotCogapsPatterns_parameters = {"Patterns": {"optional": True},
                                 "Phenotype": {"optional": True},
                                 "Phenotype_Values": {"optional": True},
                                 "Width": {"type": "number", "default": 4, "optional": True},
                                 "Subset": bool_param,
                                 "Visualization_Coordinates": vis_coordinates_param,
                                 "output_var": {"hide": True}}

def PlotCogapsPatterns(AnnData_File, Visualization_Coordinates, Patterns, Phenotype, Phenotype_Values, Subset, Width):
    # check that a cogaps result file has been merged into this AnnData object
    adata = sc.read(AnnData_File)
    if 'nPatterns' not in adata.uns.keys():
        print("Error: not cogaps patterns found")
        return

    # process Patterns argument
    if Patterns == '':
        Patterns = [str(i+1) for i in range(adata.uns['nPatterns'])]
    else:
        Patterns = str(Patterns).split(',')
        Patterns = [x.strip() for x in Patterns]

    # process Phenotype_Values argument
    if Phenotype_Values == '':
        Phenotype_Values = []
    else:
        Phenotype_Values = str(Phenotype_Values).split(',')
        Phenotype_Values = [x.strip() for x in Phenotype_Values]
    
    # check if we are subsetting to the given phenotype values
    if Subset and Phenotype == '':
        print("Must provide Phenotype argument when subsetting")
        return
    if Subset and len(Phenotype_Values) == 0:
        print("Must provide Phenotype_Values when subsetting")
        return
    if Subset:
        index = adata.obs[Phenotype] == Phenotype_Values[0]
        for pheno_val in Phenotype_Values:
            index |= adata.obs[Phenotype] == pheno_val
        adata = adata[index]
       
    # create list of each plot we are making
    colors = ['cogaps.pattern.' + p for p in Patterns]
    if Phenotype != '' and len(Phenotype_Values) == 0:
        colors += [Phenotype] # plot entire phenotype if no specific values requested
    
    # create a plot highligthing each phenotype value
    if (Subset and len(Phenotype_Values) > 1) or (not Subset and len(Phenotype_Values) > 0):
        all_phenotypes = [str(x) for x in list(sorted(set(adata.obs[Phenotype])))]
        for pheno_val in Phenotype_Values:
            name = Phenotype + ": " + str(pheno_val)
            adata.obs[name] = adata.obs[Phenotype]
            adata.uns[name + "_colors"] = ['gray' for x in all_phenotypes]
            if pheno_val not in all_phenotypes:
                print("Error:", str(pheno_val), " - phenotype value not found")
                return
            adata.uns[name + "_colors"][all_phenotypes.index(str(pheno_val))] = 'red'
            colors += [name]
        
    # create plot
    if Visualization_Coordinates == 'UMAP':
        sc.pl.umap(adata, color=colors, palette=None, ncols=Width)
    else:
        sc.pl.tsne(adata, color=colors, palette=None, ncols=Width)

#### KruskalWallisPatternTest ####
        
KruskalWallisPatternTest_description = "Runs a Kruskal-Wallis test for a categorical phenotype across all patterns."
KruskalWallisPatternTest_parameters = {"output_var": {"hide": True}}

def KruskalWallisPatternTest(AnnData_File, Phenotype):
    adata = sc.read(AnnData_File)

    # make a list of the indices for each value of the phenotype
    ndx = []
    for val in set(adata.obs[Phenotype]):
        ndx.append([i for i, x in enumerate(adata.obs[Phenotype]) if x == val])

    # for each pattern, split up values based on the associated cell phenotype and run KW
    stats = []
    for i in range(adata.uns['nPatterns']):
        patternByPhenotype = []
        for indices in ndx:
            patternByPhenotype.append(adata.obs['cogaps.pattern.' + str(i+1)][indices])
        kw_test = ss.kruskal(*patternByPhenotype)
        stats.append([i+1, kw_test.statistic, kw_test.pvalue])

    # sort, print, and plot the results of the test
    stats.sort(key= lambda x: x[1], reverse=True)
    print(pd.DataFrame(stats, columns=['pattern', 'Kruskal-Wallis Statistic', 'p-value']))
    plt.boxplot([s[1] for s in stats])
    plt.show()
    
#### DunnTest ####
    
DunnTest_description = "Runs a Dunn test for a categorical phenotype on a specific pattern."
DunnTest_parameters = {"output_var": {"hide": True}}

def DunnTest(AnnData_File, Pattern, Phenotype):
    adata = sc.read(AnnData_File)
    pattern_values = adata.obs['cogaps.pattern.' + str(Pattern)]
    ndx = []
    for val in sorted(set(adata.obs[Phenotype])):
        ndx.append([i for i, x in enumerate(adata.obs[Phenotype]) if x == val])
    %R library(dunn.test)
    %R -i ndx ndx <- lapply(ndx, unlist)
    %R -i pattern_values patternByPhenotype <- lapply(ndx, function(n) pattern_values[n])
    %R dunn.test(patternByPhenotype, kw=TRUE, wrap=TRUE)
            
#### PlotCellTypePatternCorrelation ####
        
PlotCellTypePatternCorrelation_description = "Plot a heatmap of corrlation between cell types and patterns."
PlotCellTypePatternCorrelation_parameters = {"output_var":{"hide": True,},}

def PlotCellTypePatternCorrelation(AnnData_File):
    adata = sc.read(AnnData_File)
    cell_types = adata.obs['cell_type'].unique()
    cell_types = [x for x in cell_types if x != 'Unknown']
    cell_type_index = {}
    for ct in cell_types:
        cell_type_index[ct] = adata.obs['cell_type'] == ct
        
    # find the correlation for each cell type and each pattern
    known = adata.obs['cell_type'] != 'Unknown'
    cell_type_pattern_corr = []
    for ct in cell_types:
        corr = []
        for i in range(adata.uns['nPatterns']):
            corr.append(ss.pointbiserialr(cell_type_index[ct][known], adata.obs['cogaps.pattern.' + str(i+1)][known]).correlation)
        cell_type_pattern_corr.append(corr)
    
    # form a heatmap of the correlations
    cell_type_pattern_corr = np.array(cell_type_pattern_corr)
    ax = sb.heatmap(cell_type_pattern_corr, linewidth=1, yticklabels=cell_types, xticklabels=['pattern '+str(i+1) for i in range(adata.uns['nPatterns'])])
    plt.show()

#### CogapsCellTypeIdentification ####
    
CogapsCellTypeIdentification_description = "Run cell type identification with Cogaps patterns."
CogapsCellTypeIdentification_parameters = {"output_var": {"hide": True}}

def CogapsCellTypeIdentification(AnnData_File, Marker_Genes_File):
    adata = sc.read(AnnData_File)
    
    # read marker file
    cell_types = []
    marker_genes = []
    with open(Marker_Genes_File) as f:
        for line in f:
            if line[0] == '>':
                cell_types.append(line[1:].split('\n')[0])
            if 'expressed:' in line:
                stripped = line.split('expressed:')[1].split('\n')[0]
                stripped = stripped.replace(' ', '')
                marker_genes.append(stripped.split(','))
    if len(cell_types) != len(marker_genes):
        print("Error: invalid marker file")
        return         
        
    # create marker dictionary
    markers = {}
    for n in range(len(cell_types)):
        markers[cell_types[n]] = marker_genes[n]
        
    # print top 3 patterns by rank for each marker gene
    for ct in markers.keys():
        ndx = [i for i, x in enumerate(adata.var['gene_short_name']) if x in markers[ct]]
        print("\nmarker genes found for", ct)
        for n in ndx:
            gene_name = adata.var['gene_short_name'][n]
            ranks = [adata.var[k][n] for k in adata.var_keys() if 'rank' in k] # get pattern ranks
            ranks = np.array(ranks)
            top3 = np.argsort(ranks)[:3]
            print(gene_name, "\ttop 3 patterns:", top3 + 1, "\tranks:", ranks[top3])
    
#### Code for this UI Cell ####

def LoadFunctions():
    print("Functions Loaded")

genepattern.GPUIBuilder(LoadFunctions,
                        description="Loads all functions used in this notebook.",
                        parameters={"output_var":{"hide": True}})

UIBuilder(description='Loads all functions used in this notebook.', function_import='LoadFunctions', name='Loa…

# Download Raw Data
<a href=#Table-of-Contents>back to top</a>  

The first thing we must do is download the raw data. We'll also want to create a directory to store the data and all of our work in this notebook. The next cell creates a directory called "ica_cord_blood". For the rest of the notebook, all the files we create will be stored here.

<div class="alert alert-info">
<b style="font-size:120%;">CreateDirectory</b><br>    
<b>Directory Name</b> ica_cord_blood
</div>

In [12]:
genepattern.GPUIBuilder(CreateDirectory,
                        description=CreateDirectory_description,
                        parameters=CreateDirectory_parameters)

UIBuilder(description='Create a new directory.', function_import='CreateDirectory', name='CreateDirectory', pa…

We are studying the umbilical cord blood subset of the Census of Immune Cells dataset, provided as a [preview data set](https://preview.data.humancellatlas.org/) from the Human Cell Atlas project. We will first download the raw file ([download link](https://s3.amazonaws.com/preview-ica-expression-data/ica_cord_blood_h5.h5)) so that we can preview the contents.

<div class="alert alert-info">
<b style="font-size:120%;">DownloadFile</b><br>    
<b>URL</b> https://s3.amazonaws.com/preview-ica-expression-data/ica_cord_blood_h5.h5<br>
<b>File Name</b> ica_cord_blood/ica_cord_blood_h5.h5
</div>

In [13]:
genepattern.GPUIBuilder(DownloadFile,
                        description=DownloadFile_description,
                        parameters=DownloadFile_parameters)

UIBuilder(description='Download a file from the given URL.', function_import='DownloadFile', name='DownloadFil…

The downloaded file is in an HDF5 format, which is a commonly used format for single-cell data. The next cell allows us to see the contents of this file.

<div class="alert alert-info">
<b style="font-size:120%;">InspectHDF5</b><br>    
<b>File Name</b> ica_cord_blood/ica_cord_blood_h5.h5
</div>

In [14]:
genepattern.GPUIBuilder(InspectHDF5,
                        description=InspectHDF5_description,
                        parameters=InspectHDF5_parameters)

UIBuilder(description='List the contents of an HDF5 file.', function_import='InspectHDF5', name='InspectHDF5',…

In [15]:
genepattern.GPUIBuilder(InspectHDF5,
                        description=InspectHDF5_description,
                        parameters=InspectHDF5_parameters)

UIBuilder(description='List the contents of an HDF5 file.', function_import='InspectHDF5', name='InspectHDF5',…

In [ ]:
ica_cord_blood/

<div class="well well-sm">
<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Edit this cell to insert your text here.
</div>
</div>

This file is stored in the [10x HDF5 format](https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/advanced/h5_matrices). Before we get started on the analysis, we'll want to convert this to a more friendly file format. We'll be using the Python package **scanpy** throughout this analysis and so it would make sense to store our data in the **AnnData** format, ".h5ad". This format will work automatically with all of **scanpy**'s functionality. This file format is important since we'll need to outsource a lot of the analysis done in the notebook to GenePattern modules. These modules run on an external server and so we need a file that we can upload. The general workflow will be to make some changes to an ".h5ad" file in the notebook, send it to a module for some more heavy duty computation, and then pull it back in the notebook for more analysis. Since we repeat this so much, it makes sense to have a file format that's easy to work with.

We can use the *ScanpyUtilities* module to convert a 10x formatted HDF5 file to the **AnnData** format. We'll need to pass the name of the dataset, which we can see from the "InspectHDF5" cell is "GRCh38". Follow the instructions below to download and convert the raw data.

<div class="alert alert-info">
<h3 style="margin-top: 0;"> ScanpyUtilities Module Instructions - Converting to h5ad <i class="fa fa-info-circle"></i></h3>
<ol>
    <li>Under "Input and Output" set <b>data file</b> to <a href="data" target="_blank">"https://s3.amazonaws.com/preview-ica-expression-data/ica_cord_blood_h5.h5"</a> </li>
    <li>Under "Input and Output" set <b>output basename</b> to "full_data"</li>
    <li>Under "Input and Output" set <b>genome</b> to "GRCh38"</li>
    <li>Everything else should be set to it's default (either blank or "no")</li>
</ol>
Note: do not include the quotes when setting parameters
</div>

In [2]:
scanpyutilities_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00382')
scanpyutilities_job_spec = scanpyutilities_task.make_job_spec()
scanpyutilities_job_spec.set_parameter("data.file", "https://s3.amazonaws.com/preview-ica-expression-data/ica_cord_blood_h5.h5")
scanpyutilities_job_spec.set_parameter("output.basename", "full_data")
scanpyutilities_job_spec.set_parameter("annotate", "0")
scanpyutilities_job_spec.set_parameter("cells.min.counts", "")
scanpyutilities_job_spec.set_parameter("cells.max.counts", "")
scanpyutilities_job_spec.set_parameter("cells.min.genes", "")
scanpyutilities_job_spec.set_parameter("cells.max.genes", "")
scanpyutilities_job_spec.set_parameter("genes.min.counts", "")
scanpyutilities_job_spec.set_parameter("genes.max.counts", "")
scanpyutilities_job_spec.set_parameter("genes.min.cells", "")
scanpyutilities_job_spec.set_parameter("genes.max.cells", "")
scanpyutilities_job_spec.set_parameter("normalize", "0")
scanpyutilities_job_spec.set_parameter("n.high.variance.genes", "")
scanpyutilities_job_spec.set_parameter("compute.umap", "0")
scanpyutilities_job_spec.set_parameter("compute.tsne", "0")
scanpyutilities_job_spec.set_parameter("genome", "GRCh38")
scanpyutilities_job_spec.set_parameter("job.memory", "512Gb")
scanpyutilities_job_spec.set_parameter("job.queue", "gpbeta-default")
scanpyutilities_job_spec.set_parameter("job.cpuCount", "16")
scanpyutilities_job_spec.set_parameter("job.walltime", "24:00:00")
genepattern.display(scanpyutilities_task)

job99219 = gp.GPJob(genepattern.session.get(0), 99219)
genepattern.display(job99219)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00382')

GPJobWidget(job_number=99219)

Once the module is finished, we'll see that it has created "full_data.h5ad" which is our converted data set. In order to work with this file, we first need to download it back to the notebook.

<div class="alert alert-info">
<b style="font-size:120%;">DownloadJobResultFile</b><br>    
<b>Job File Url</b> Select "full_data.h5ad" from the dropdown menu.<br>
<b>File Name</b> ica_cord_blood/full_data.h5ad
</div>

In [16]:
genepattern.GPUIBuilder(DownloadJobResultFile,
                        description=DownloadJobResultFile_description,
                        parameters=DownloadJobResultFile_parameters)

UIBuilder(description='Download file from a GenePattern module job result.', function_import='DownloadJobResul…

# Add Preliminary Annotations
<a href=#Table-of-Contents>back to top</a>  

In this section we add some preliminary annotations to the data. These are things like which donor a sample comes from and how many expressed genes it has. These annotations will prove useful for filtering out bad samples and doing some preliminary visualizations of the data. Some of these annotations we can make without referring to the actual count data, but some  things, such as number of expressed genes per sample, require accessing the full data. This is something we can't do without exceeding the memory capacity of the notebook. We will use the *ScanpyUtilities* module for making these annotations.

## Examining Data
<a href=#Table-of-Contents>back to top</a>  

Before we can get started addding annotations to the data, we first need to take a look at what information we have. These next cells print some information from a random sample of genes (features) and cells (samples).

<div class="alert alert-info">
<b style="font-size:120%;">ViewRandomSubset</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data.h5ad<br>
<b>Num Values</b> 10<br>
<b>Which Axis</b> samples
</div>

In [17]:
genepattern.GPUIBuilder(ViewRandomSubset,
                        description=ViewRandomSubset_description,
                        parameters=ViewRandomSubset_parameters)

UIBuilder(description='View a random subset of the AnnData object.', function_import='ViewRandomSubset', name=…

<div class="alert alert-info">
<b style="font-size:120%;">ViewRandomSubset</b><br>
<b>AnnData File</b> ica_cord_blood/full_data.h5ad<br>
<b>Num Values</b> 10<br>
<b>Which Axis</b> features
</div>

In [18]:
genepattern.GPUIBuilder(ViewRandomSubset,
                        description=ViewRandomSubset_description,
                        parameters=ViewRandomSubset_parameters)

UIBuilder(description='View a random subset of the AnnData object.', function_import='ViewRandomSubset', name=…

So it looks like we have barcodes for each cell and gene names/ids for each gene. Notice that we are also getting a warning about variable names not being unique. This is because the index for the genes is the gene name, which appartently has some repeats. It would be a good idea to use the ids for the index instead since those are unique. Also for some of the analysis later on, we will require a column named "gene_short_name" to contain the names of each gene. We can accomplish both of these tasks with the following cell.

<div class="alert alert-info">
<b style="font-size:120%;">SwapIndexAndColumn</b><br>
<b>AnnData File</b> ica_cord_blood/full_data.h5ad<br>
<b>Index Column</b> gene_ids<br>
<b>New Column Name</b> gene_short_name<br>
<b>Which Axis</b> features
</div>

In [19]:
genepattern.GPUIBuilder(SwapIndexAndColumn,
                        description=SwapIndexAndColumn_description,
                        parameters=SwapIndexAndColumn_parameters)

UIBuilder(description='Create a new index for the data using an existing column.', function_import='SwapIndexA…

The next cell shows the new and improved layout for the gene data - this time with no warnings.

<div class="alert alert-info">
<b style="font-size:120%;">ViewRandomSubset</b><br>
<b>AnnData File</b> ica_cord_blood/full_data.h5ad<br>
<b>Num Values</b> 10<br>
<b>Which Axis</b> features
</div>

In [20]:
genepattern.GPUIBuilder(ViewRandomSubset,
                        description=ViewRandomSubset_description,
                        parameters=ViewRandomSubset_parameters)

UIBuilder(description='View a random subset of the AnnData object.', function_import='ViewRandomSubset', name=…

## Experimental Conditions
<a href=#Table-of-Contents>back to top</a>  

We next want to add some information to our cells since right now all we have are the barcodes, embedded in the barcodes are the donor and channel ids, which are going to be important for analysis. If we take a look at a single barcode, e.g. **MantonCB5_HiSeq_7-ACGATACTCACCTTAT-1**, we can see that the donor id (5) is located at the 8th position and the channel id (7) is located at the 16th position. Furthermore, both ids are single digits so this will always be the case. This next cell uses [regular expressions](https://www.regular-expressions.info/) to parse the variable names and return the parsed values as a variable. We can use it to extract the ids in the 8th and 16th position.

<div class="alert alert-info">
<b style="font-size:120%;">RegexVariableNames</b><br>
<b>AnnData File</b> ica_cord_blood/full_data.h5ad<br>
<b>Pattern</b> '(?<=^.{8}).'<br>
<b>Which Axis</b> samples<br>
<b>variable name for regex output</b> donor_id
</div>

In [21]:
genepattern.GPUIBuilder(RegexVariableNames,
                        description=RegexVariableNames_description,
                        parameters=RegexVariableNames_parameters)

UIBuilder(description='Parse AnnData column using regex pattern.', function_import='RegexVariableNames', name=…

<div class="alert alert-info">
<b style="font-size:120%;">View</b><br>
<b>Variable Name</b> donor_id
</div>

In [22]:
genepattern.GPUIBuilder(View,
                        description=View_description,
                        parameters=View_parameters)

UIBuilder(description='View contents of variable.', function_import='View', name='View', params=[{'name': 'Var…

<div class="alert alert-info">
<b style="font-size:120%;">RegexVariableNames</b><br>
<b>AnnData File</b> ica_cord_blood/full_data.h5ad<br>
<b>Pattern</b> '(?<=^.{16}).'<br>
<b>Which Axis</b> samples<br>
<b>variable name for regex output</b> channel_id
</div>

In [23]:
genepattern.GPUIBuilder(RegexVariableNames,
                        description=RegexVariableNames_description,
                        parameters=RegexVariableNames_parameters)

UIBuilder(description='Parse AnnData column using regex pattern.', function_import='RegexVariableNames', name=…

<div class="alert alert-info">
<b style="font-size:120%;">View</b><br>
<b>Variable Name</b> channel_id
</div>

In [24]:
genepattern.GPUIBuilder(View,
                        description=View_description,
                        parameters=View_parameters)

UIBuilder(description='View contents of variable.', function_import='View', name='View', params=[{'name': 'Var…

Now that we have variables with the donor and channel ids, we want to add them to the cell data. The next cell allows us to create new columns in a **AnnData** object.

<div class="alert alert-info">
<b style="font-size:120%;">AddColumnToData</b><br>
<b>AnnData File</b> ica_cord_blood/full_data.h5ad<br>
<b>Column Name</b> donor<br>
<b>Values</b> donor_id<br>
<b>Which Axis</b> samples<br>
</div>

In [25]:
genepattern.GPUIBuilder(AddColumnToData,
                        description=AddColumnToData_description,
                        parameters=AddColumnToData_parameters)

UIBuilder(description='Adds a column to an AnnData object.', function_import='AddColumnToData', name='AddColum…

<div class="alert alert-info">
<b style="font-size:120%;">AddColumnToData</b><br>
<b>AnnData File</b> ica_cord_blood/full_data.h5ad<br>
<b>Column Name</b> channel<br>
<b>Values</b> channel_id<br>
<b>Which Axis</b> samples<br>
</div>

In [26]:
genepattern.GPUIBuilder(AddColumnToData,
                        description=AddColumnToData_description,
                        parameters=AddColumnToData_parameters)

UIBuilder(description='Adds a column to an AnnData object.', function_import='AddColumnToData', name='AddColum…

Now we can take a look at the new and improved cell data.

<div class="alert alert-info">
<b style="font-size:120%;">ViewRandomSubset</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data.h5ad<br>
<b>Num Values</b> 10<br>
<b>Which Axis</b> samples
</div>

In [27]:
genepattern.GPUIBuilder(ViewRandomSubset,
                        description=ViewRandomSubset_description,
                        parameters=ViewRandomSubset_parameters)

UIBuilder(description='View a random subset of the AnnData object.', function_import='ViewRandomSubset', name=…

## Count Information
<a href=#Table-of-Contents>back to top</a>  

Now we want to add some count information to the data. Specifically the number of counts expressed in each gene and cell, as well as the total number of cells expressed in each gene and vice versa. We will use the *ScanpyUtilities* module for this since we need access to the full data set and there is not enough memory to do that in the notebook. First we need to upload our data set to the GenePattern server so that we can use it as input for the module.

<div class="alert alert-info">
<b style="font-size:120%;">UploadFileToGenePatternServer</b><br>    
<b>Local File</b> ica_cord_blood/full_data.h5ad<br>
<b>Remote File Name</b> ica_cord_blood_full_data.h5ad<br>
<b>variable name for url</b> data_url
</div>

In [28]:
genepattern.GPUIBuilder(UploadFileToGenePatternServer,
                        description=UploadFileToGenePatternServer_description,
                        parameters=UploadFileToGenePatternServer_parameters)

UIBuilder(description='Upload the local file onto the GenePattern server.', function_import='UploadFileToGeneP…

<div class="alert alert-info">
<h3 style="margin-top: 0;"> ScanpyUtilities Module Instructions - Annotating the Data <i class="fa fa-info-circle"></i></h3>
<ol>
    <li>Under "Input and Output" set <b>data file</b> to "{{ data_url }}", when using the {{ x }} syntax it will expand the value of the variable as a string</li>
    <li>Under "Input and Output" set <b>output basename</b> to "full_data"</li>
    <li>Under "Annotation" set <b>annotate</b> to "yes"</li>
    <li>Everything else should be set to it's default (either blank or "no")</li>
</ol>
Note: do not include the quotes when setting parameters
</div>

In [3]:
scanpyutilities_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00382')
scanpyutilities_job_spec = scanpyutilities_task.make_job_spec()
scanpyutilities_job_spec.set_parameter("data.file", "{{ uploaded_url }}")
scanpyutilities_job_spec.set_parameter("output.basename", "full_data")
scanpyutilities_job_spec.set_parameter("annotate", "1")
scanpyutilities_job_spec.set_parameter("cells.min.counts", "")
scanpyutilities_job_spec.set_parameter("cells.max.counts", "")
scanpyutilities_job_spec.set_parameter("cells.min.genes", "")
scanpyutilities_job_spec.set_parameter("cells.max.genes", "")
scanpyutilities_job_spec.set_parameter("genes.min.counts", "")
scanpyutilities_job_spec.set_parameter("genes.max.counts", "")
scanpyutilities_job_spec.set_parameter("genes.min.cells", "")
scanpyutilities_job_spec.set_parameter("genes.max.cells", "")
scanpyutilities_job_spec.set_parameter("normalize", "0")
scanpyutilities_job_spec.set_parameter("n.high.variance.genes", "")
scanpyutilities_job_spec.set_parameter("compute.umap", "0")
scanpyutilities_job_spec.set_parameter("compute.tsne", "0")
scanpyutilities_job_spec.set_parameter("genome", "")
scanpyutilities_job_spec.set_parameter("job.memory", "32 Gb")
scanpyutilities_job_spec.set_parameter("job.queue", "gpbeta-default")
scanpyutilities_job_spec.set_parameter("job.cpuCount", "4")
scanpyutilities_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(scanpyutilities_task)

job100233 = gp.GPJob(genepattern.session.get(0), 100233)
genepattern.display(job100233)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00382')

GPJobWidget(job_number=100233)

<div class="alert alert-info">
<b style="font-size:120%;">DownloadJobResultFile</b><br>    
<b>Job File Url</b> Select "full_data_annotated.h5ad" from the dropdown menu.<br>
<b>File Name</b> ica_cord_blood/full_data_annotated.h5ad
</div>

In [29]:
genepattern.GPUIBuilder(DownloadJobResultFile,
                        description=DownloadJobResultFile_description,
                        parameters=DownloadJobResultFile_parameters)

UIBuilder(description='Download file from a GenePattern module job result.', function_import='DownloadJobResul…

# Filter Low Quality Samples
<a href=#Table-of-Contents>back to top</a>  

In this section we will be filtering out the low quality samples and the lowly expressed genes. This part isn't an exact science so we will be making several plots of our count data in order to get a sense of the distribution. From these plots we can make some decisions about filtering.

## Cell Filter  - Count Threshold
<a href=#Table-of-Contents>back to top</a>  

In this section we come up with a minimum and maximum threshold for the number of counts each cell must have to remain in the analysis. We plot the distribution of the total number of counts per sample and zoom in at each tail. From these plots it seems that a reasonable minimum threshold would be 1000 counts since the distribution of cells below 1000 seems artificially dense. For a maximum threshold we choose 20000 since cells that have more counts than this seem to be outliers.

<div class="alert alert-info">
<b style="font-size:120%;">Histogram</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_annotated.h5ad<br>
<b>Plot Variable</b> n_counts<br>
<b>Which Axis</b> samples<br>
<b>Lower Bound</b> -- leave blank --<br>
<b>Upper Bound</b> -- leave blank --
</div>

In [30]:
genepattern.GPUIBuilder(Histogram,
                        description=Histogram_description,
                        parameters=Histogram_parameters)

UIBuilder(description='Create a histogram of an AnnData object.', function_import='Histogram', name='Histogram…

<div class="alert alert-info">
<b style="font-size:120%;">Histogram</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_annotated.h5ad<br>
<b>Plot Variable</b> n_counts<br>
<b>Which Axis</b> samples<br>
<b>Lower Bound</b> -- leave blank --<br>
<b>Upper Bound</b> 2000
</div>

In [31]:
genepattern.GPUIBuilder(Histogram,
                        description=Histogram_description,
                        parameters=Histogram_parameters)

UIBuilder(description='Create a histogram of an AnnData object.', function_import='Histogram', name='Histogram…

<div class="alert alert-info">
<b style="font-size:120%;">Histogram</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_annotated.h5ad<br>
<b>Plot Variable</b> n_counts<br>
<b>Which Axis</b> samples<br>
<b>Lower Bound</b> 10000<br>
<b>Upper Bound</b> -- leave blank --
</div>

In [32]:
genepattern.GPUIBuilder(Histogram,
                        description=Histogram_description,
                        parameters=Histogram_parameters)

UIBuilder(description='Create a histogram of an AnnData object.', function_import='Histogram', name='Histogram…

## Cell Filter - Gene Threshold
<a href=#Table-of-Contents>back to top</a>  

Using the same approach as above, we can set some thresholds for the number of genes that a cell needs to express to remain in the analysis. We can see that the density of samples with less than 500 genes expressed is artificially dense, so we choose this for a minimum threshold. We don't choose any maximum threshold since there isn't a clear point to make it at.

<div class="alert alert-info">
<b style="font-size:120%;">Histogram</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_annotated.h5ad<br>
<b>Plot Variable</b> n_genes<br>
<b>Which Axis</b> samples<br>
<b>Lower Bound</b> -- leave blank --<br>
<b>Upper Bound</b> -- leave blank --
</div>

In [33]:
genepattern.GPUIBuilder(Histogram,
                        description=Histogram_description,
                        parameters=Histogram_parameters)

UIBuilder(description='Create a histogram of an AnnData object.', function_import='Histogram', name='Histogram…

<div class="alert alert-info">
<b style="font-size:120%;">Histogram</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_annotated.h5ad<br>
<b>Plot Variable</b> n_counts<br>
<b>Which Axis</b> samples<br>
<b>Lower Bound</b> -- leave blank --<br>
<b>Upper Bound</b> 1000
</div>

In [34]:
genepattern.GPUIBuilder(Histogram,
                        description=Histogram_description,
                        parameters=Histogram_parameters)

UIBuilder(description='Create a histogram of an AnnData object.', function_import='Histogram', name='Histogram…

<div class="alert alert-info">
<b style="font-size:120%;">Histogram</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_annotated.h5ad<br>
<b>Plot Variable</b> n_genes<br>
<b>Which Axis</b> samples<br>
<b>Lower Bound</b> 2000<br>
<b>Upper Bound</b> -- leave blank --
</div>

In [35]:
genepattern.GPUIBuilder(Histogram,
                        description=Histogram_description,
                        parameters=Histogram_parameters)

UIBuilder(description='Create a histogram of an AnnData object.', function_import='Histogram', name='Histogram…

## Use the ScanpyUtilities Module to Filter
<a href=#Table-of-Contents>back to top</a>  

Now it's time to apply our proposed filter. Since this requires accessing the full data we will have to use the *ScanpyUtilities* module. Since we haven't modified our data from the previous job, we can actually pass the job result directly to the input of this module instead of uploading our local file. Note here that we also filter out any genes that aren't expressed in at least 20 cells. This is just a minimum baseline for determining relevant genes. This isn't necessarily a critical threshold since most of our downstream analysis will happen on a highly variable subset of genes anyways.


<div class="alert alert-info">
<h3 style="margin-top: 0;"> ScanpyUtilities Module Instructions - Filtering the Data <i class="fa fa-info-circle"></i></h3>
<ol>
    <li>Under "Input and Output" select "full_data_annotated" from the <b>data file</b> dropdown</li>
    <li>Under "Input and Output" set <b>output basename</b> to "full_data"</li>
    <li>Under "Cell Filtering" set <b>cells min counts</b> to 1000</li>
    <li>Under "Cell Filtering" set <b>cells max counts</b> to 20000</li>
    <li>Under "Cell Filtering" set <b>cells min genes</b> to 500</li>
    <li>Under "Gene Filtering" set <b>genes min cells</b> to 20</li>
    <li>Everything else should be set to it's default (either blank or "no")</li>
</ol>
Note: do not include the quotes when setting parameters
</div>

In [4]:
scanpypreprocess_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00382')
scanpypreprocess_job_spec = scanpypreprocess_task.make_job_spec()
scanpypreprocess_job_spec.set_parameter("data.file", "https://cloud.genepattern.org/gp/jobResults/100233/full_data_annotated.h5ad")
scanpypreprocess_job_spec.set_parameter("group.name", "GRCh38")
scanpypreprocess_job_spec.set_parameter("output.filename", "<data.file_basename>")
scanpypreprocess_job_spec.set_parameter("annotate", "0")
scanpypreprocess_job_spec.set_parameter("cells.min.counts", "1000")
scanpypreprocess_job_spec.set_parameter("cells.max.counts", "20000")
scanpypreprocess_job_spec.set_parameter("cells.min.genes", "500")
scanpypreprocess_job_spec.set_parameter("cells.max.genes", "")
scanpypreprocess_job_spec.set_parameter("genes.min.counts", "")
scanpypreprocess_job_spec.set_parameter("genes.max.counts", "")
scanpypreprocess_job_spec.set_parameter("genes.min.cells", "20")
scanpypreprocess_job_spec.set_parameter("genes.max.cells", "")
scanpypreprocess_job_spec.set_parameter("batch.correct.and.normalize", "False")
scanpypreprocess_job_spec.set_parameter("job.memory", "16 Gb")
scanpypreprocess_job_spec.set_parameter("job.queue", "gp-new-beta")
scanpypreprocess_job_spec.set_parameter("job.cpuCount", "4")
scanpypreprocess_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(scanpypreprocess_task)

job100296 = gp.GPJob(genepattern.session.get(0), 100296)
genepattern.display(job100296)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00382')

GPJobWidget(job_number=100296)

Notice that there are two result files from this job, one for the cell filter and one for the gene filter. When running multiple sections of the *ScanpyUtilities* module, there will be a result file for each section. The sections are run in order, so in this case we want to download the gene filter as that will contain the results from both filters.

<div class="alert alert-info">
<b style="font-size:120%;">DownloadJobResultFile</b><br>
<b>Job File Url</b> select "full_data_gene_filter.h5ad" from the dropdown<br>
<b>File Name</b> ica_cord_blood/full_data_filtered.h5ad
</div>

In [36]:
genepattern.GPUIBuilder(DownloadJobResultFile,
                        description=DownloadJobResultFile_description,
                        parameters=DownloadJobResultFile_parameters)

UIBuilder(description='Download file from a GenePattern module job result.', function_import='DownloadJobResul…

## Remaining Cells
<a href=#Table-of-Contents>back to top</a>  

In this section we'll visualize the overall distribution of the remaining cells to see how well the filter worked. First we look at the original data. We make a scatter plot of the number of counts vs the number of genes expressed in each cell. This is informative since we expect a roughly linear trend so outliers will be easy to spot if they lie outside the main group. Interestingly, in the original data is seems like there is an entire sub-population of cells that have a high number of counts relative to how many genes they express. These are likely outliers and should be filtered out for this analysis

We also take a look at a violin plot of the number of counts for each cell, grouped by donor. This gives us a rough look at the quality of samples we have from each donor. In this case, there is alot of noise of the quality varies quite a bit between each donor.

<div class="alert alert-info">
<b style="font-size:120%;">CellScatterPlot</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_annotated.h5ad<br>
<b>x variable</b> n_counts<br>
<b>y variable</b> n_genes<br>
</div>

In [37]:
genepattern.GPUIBuilder(CellScatterPlot,
                    description=CellScatterPlot_description,
                    parameters=CellScatterPlot_parameters)  

UIBuilder(description='Create a scatter plot of cell features.', function_import='CellScatterPlot', name='Cell…

<div class="alert alert-info">
<b style="font-size:120%;">ViolinPlot</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_annotated.h5ad<br>
<b>Plot Variable</b> n_counts<br>
<b>Group Variable</b> donor
</div>

In [38]:
genepattern.GPUIBuilder(ViolinPlot,
                    description=ViolinPlot_description,
                    parameters=ViolinPlot_parameters)

UIBuilder(description='Create a violin plot from an AnnData object stored in a file.', function_import='Violin…

Now we do the same visualizations with the filtered data. We can see the the filters eliminated most of the outlier population in the scatter plot, and drastically smoothed out the distribution of counts for each donor. At this point, we can comfortably proceed on with the analysis.

<div class="alert alert-info">
<b style="font-size:120%;">CellScatterPlot</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_filtered.h5ad<br>
<b>x variable</b> n_counts<br>
<b>y variable</b> n_genes<br>
</div>

In [39]:
genepattern.GPUIBuilder(CellScatterPlot,
                    description=CellScatterPlot_description,
                    parameters=CellScatterPlot_parameters)

UIBuilder(description='Create a scatter plot of cell features.', function_import='CellScatterPlot', name='Cell…

<div class="alert alert-info">
<b style="font-size:120%;">ViolinPlot</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_filtered.h5ad<br>
<b>Plot Variable</b> n_counts<br>
<b>Group Variable</b> donor
</div>

In [40]:
genepattern.GPUIBuilder(ViolinPlot,
                    description=ViolinPlot_description,
                    parameters=ViolinPlot_parameters)

UIBuilder(description='Create a violin plot from an AnnData object stored in a file.', function_import='Violin…

## Subset Data for Further Analysis
<a href=#Table-of-Contents>back to top</a>

At this point, we've filtered our data down to about 250,000 cells and 20,000 genes. However the 250,000 cells come from 8 donors. It is reasonable to expect a significant batch effect from these 8 donors and there are several methods for accounting for this. In this version of the notebook however, accounting for batch effects is considered out of scope. We will continue the analysis by selecting a single donor to analyze, and in future versions of the notebook we will discuss strategies for dealing with batch effects. In this section we subset down to donor 5, which has the most remaining cells after the filtering stage. Since this will remove so many cells, it is possible that some of the genes will now have zero expression and so we must do another round of gene filtering. This is done automatically in the "SubsetData" cell.

<div class="alert alert-info">
<b style="font-size:120%;">CountCellByGroup</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_filtered.h5ad<br>
<b>Group Variable</b> donor<br>
</div>

In [41]:
genepattern.GPUIBuilder(CountCellsByGroup,
                    description=CountCellsByGroup_description,
                    parameters=CountCellsByGroup_parmeters)

UIBuilder(description='Get cell counts split by group.', function_import='CountCellsByGroup', name='CountCells…

<div class="alert alert-info">
<b style="font-size:120%;">SubsetData</b><br>    
<b>AnnData File</b> ica_cord_blood/full_data_filtered.h5ad<br>
<b>Output File</b> ica_cord_blood/donor_5_filtered.h5ad<br>
<b>Group Variable</b> donor<br>
<b>Group Value</b> 5
</div>

In [42]:
genepattern.GPUIBuilder(SubsetData,
                    description=SubsetData_description,
                    parameters=SubsetData_parameters)

UIBuilder(description='Subset a data set to only cells of a certain group.', function_import='SubsetData', nam…

# Cell Type Identification
<a href=#Table-of-Contents>back to top</a>  

In this section, we walk through a cell type identification workflow which is a key part of any single-cell analysis. Typically cell types are identified by running a clustering algorithm, and then selecting a handful of marker genes from the literature and seeing if they are differentially expressed across clusters. Often, this is done by hand, examining the genes uniquely expressed in each cluster. Recently, however, there have been several tools published to automate this process. The R package **garnett**, which comes from the same developers as **monocle**, is one such tool.

**garnett** works by training a classifier based on an input data set and a set of marker genes. In order to train a classifier, we first need to generate a marker file. The marker file contains a list of cell type definitions in text format. Each cell type definition starts with a `>` symbol and the cell type name, followed by definition lines. These definition lines begin with a keyword and a `:`. Keywords include `Expressed` and `Not expressed` followed by the relevant expression information for each cell type. Genes listed within the definition are separated by a comma. In our marker file we define several major immune cell types along with major subtype classifications. For more information about the **garnett** workflow, refer to the [documentation](https://cole-trapnell-lab.github.io/garnett/docs/).

Once we have the marker file generated we can use the *ScanpyUtilities* module to run the **garnett** pipeline. The module will add an additional `cell_type` annotation to our ".h5ad" file. We must pass the marker file and a gene annotation database. This can be either "org.Hs.eg.db" (docs [here](https://bioconductor.org/packages/release/data/annotation/html/org.Hs.eg.db.html)) or "org.Mm.eg.db" (docs [here](https://bioconductor.org/packages/release/data/annotation/html/org.Mm.eg.db.html)). This annotation database allows **garnett** to convert the gene names we've provided in the marker file to the gene ids we've used as the index in our data. It is also important to note that we must run **garnett** before the data has been normalized. 

In [109]:
## This is one way to create a text variable that we can pass to the GenerateTextFile cell.
## marker_file_text is the name of the variable we are creating, and everything inside the 
## triple quotes we be stored as the text

marker_file_text = """
>B cells
expressed: CD19, CD38, CD24, CD34

>T cells
expressed: CD3D, CD3E, CD3G

>CD4 T cells
expressed: CD4, FOXP3, IL2RA, IL7R
subtype of: T cells

>CD8 T cells
expressed: CD8A, CD8B
subtype of: T cells

>Neutrophils
expressed: ITGAM, FCGR3A, ITGB2, FCGR2A, CD44, CD55 

>NK cells
expressed: NCAM1, FCGR3A, NCR1, KLRK1, NKG7

>DCs
expressed: CD1C, CLEC6A, CLEC7A

>pDCs
expressed: CLEC4C, NRP1, IL3RA
subtype of: DCs

>monocytes/macrophages
expressed: ITGAM, CD163, CD33, ITGAX, CD14
"""

<div class="alert alert-info">
<b style="font-size:120%;">GenerateTextFile</b><br>    
<b>Text</b> marker_file_text<br>
<b>File Name</b> ica_cord_blood/markers.txt<br>
</div>

In [43]:
genepattern.GPUIBuilder(GenerateTextFile,
                    description=GenerateTextFile_description,
                    parameters=GenerateTextFile_parameters)

UIBuilder(description='Create a text file.', function_import='GenerateTextFile', name='GenerateTextFile', para…

<div class="alert alert-info">
<b style="font-size:120%;">UploadFileToGenePatternServer</b><br>    
<b>Local File</b> ica_cord_blood/donor_5_filtered.h5ad<br>
<b>Remote File Name</b> ica_cord_blood_donor_5_filtered.h5ad<br>
<b>variable name for url</b> data_url
</div>

In [44]:
genepattern.GPUIBuilder(UploadFileToGenePatternServer,
                        description=UploadFileToGenePatternServer_description,
                        parameters=UploadFileToGenePatternServer_parameters)

UIBuilder(description='Upload the local file onto the GenePattern server.', function_import='UploadFileToGeneP…

<div class="alert alert-info">
<b style="font-size:120%;">UploadFileToGenePatternServer</b><br>    
<b>Local File</b> ica_cord_blood/markers.txt<br>
<b>Remote File Name</b> ica_cord_blood_markers.txt<br>
<b>variable name for url</b> markers_url
</div>

In [45]:
genepattern.GPUIBuilder(UploadFileToGenePatternServer,
                        description=UploadFileToGenePatternServer_description,
                        parameters=UploadFileToGenePatternServer_parameters)

UIBuilder(description='Upload the local file onto the GenePattern server.', function_import='UploadFileToGeneP…

<div class="alert alert-info">
<h3 style="margin-top: 0;"> ScanpyUtilities Module Instructions - Cell Type Identification<i class="fa fa-info-circle"></i></h3>
<ol>
    <li>Under "Input and Output" set <b>data file</b> to "{{ data_url }}"</li>
    <li>Under "Input and Output" set <b>output basename</b> to "donor_5"</li>
    <li>Under "Cell Type Identification" set <b>cell type marker file</b> to "{{ markers_url }}"</li>
    <li>Under "Cell Type Identification" set <b>gene annotation database</b> to "org.Hs.eg.db"</li>
    <li>Everything else should be set to it's default (either blank or "no")</li>
</ol>
Note: do not include the quotes when setting parameters
</div>

In [5]:
scanpyutilities_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00382')
scanpyutilities_job_spec = scanpyutilities_task.make_job_spec()
scanpyutilities_job_spec.set_parameter("data.file", "{{ data_url }}")
scanpyutilities_job_spec.set_parameter("output.basename", "donor_5")
scanpyutilities_job_spec.set_parameter("annotate", "0")
scanpyutilities_job_spec.set_parameter("cells.min.counts", "")
scanpyutilities_job_spec.set_parameter("cells.max.counts", "")
scanpyutilities_job_spec.set_parameter("cells.min.genes", "")
scanpyutilities_job_spec.set_parameter("cells.max.genes", "")
scanpyutilities_job_spec.set_parameter("genes.min.counts", "")
scanpyutilities_job_spec.set_parameter("genes.max.counts", "")
scanpyutilities_job_spec.set_parameter("genes.min.cells", "")
scanpyutilities_job_spec.set_parameter("genes.max.cells", "")
scanpyutilities_job_spec.set_parameter("cell.type.marker.file", "{{ markers_url }}")
scanpyutilities_job_spec.set_parameter("normalize", "0")
scanpyutilities_job_spec.set_parameter("n.high.variance.genes", "")
scanpyutilities_job_spec.set_parameter("compute.umap", "0")
scanpyutilities_job_spec.set_parameter("compute.tsne", "0")
scanpyutilities_job_spec.set_parameter("genome", "")
scanpyutilities_job_spec.set_parameter("gene.annotation.database", "org.Hs.eg.db")
scanpyutilities_job_spec.set_parameter("job.memory", "32 Gb")
scanpyutilities_job_spec.set_parameter("job.walltime", "02:00:00")
scanpyutilities_job_spec.set_parameter("job.cpuCount", "4")
genepattern.display(scanpyutilities_task)

job99227 = gp.GPJob(genepattern.session.get(0), 99227)
genepattern.display(job99227)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00382')

GPJobWidget(job_number=99227)

# Final Pre-Processing Steps
<a href=#Table-of-Contents>back to top</a>  

Now that we have labeled the cell types with **garnett** there are just a few more preprocessing steps we need to do before running the **CoGAPS** analysis. The **CoGAPS** R package expects the data to be log normalized and also to have undergone library size normalization. Furthermore, we need to subset down to a set of highly variable genes to improve the run time of **CoGAPS**. Finally, we will want to compute the UMAP and TSNE coordinates of our data for visualizing our results.

Each of these steps can be handled completely by the *ScanpyUtilities* module. When passing parameters for multiple sections in the module, the sections are run in the order they appear in the interface. In this case, the order will be 1) normalization 2) high variance genes 3) UMAP and TSNE calculations. After each of these sections the module will write the output to a file so we can inspect the intermediate results.

<div class="alert alert-info">
<h3 style="margin-top: 0;"> ScanpyUtilities Module Instructions - Final Pre-Processing Steps<i class="fa fa-info-circle"></i></h3>
<ol>
    <li>Under "Input and Output" select "donor_5_cell_types" from the <b>data file</b> dropdown </li>
    <li>Under "Input and Output" set <b>output basename</b> to "donor_5"</li>
    <li>Under "Normalization" set <b>normalize</b> to "yes"</li>
    <li>Under "High Variance Genes" set <b>n high variance genes</b> to "3000"</li>
    <li>Under "Dimension Reduction" set <b>compute umap</b> to "yes"</li>
    <li>Under "Dimension Reduction" set <b>compute tsne</b> to "yes"</li>
    <li>Everything else should be set to it's default (either blank or "no")</li>
</ol>
Note: do not include the quotes when setting parameters
</div>

In [6]:
scanpypreprocess_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00382')
scanpypreprocess_job_spec = scanpypreprocess_task.make_job_spec()
scanpypreprocess_job_spec.set_parameter("data.file", "https://cloud.genepattern.org/gp/jobResults/99227/donor_5_cell_types.h5ad")
scanpypreprocess_job_spec.set_parameter("output.basename", "donor_5")
scanpypreprocess_job_spec.set_parameter("annotate", "0")
scanpypreprocess_job_spec.set_parameter("cells.min.counts", "")
scanpypreprocess_job_spec.set_parameter("cells.max.counts", "")
scanpypreprocess_job_spec.set_parameter("cells.min.genes", "")
scanpypreprocess_job_spec.set_parameter("cells.max.genes", "")
scanpypreprocess_job_spec.set_parameter("genes.min.counts", "")
scanpypreprocess_job_spec.set_parameter("genes.max.counts", "")
scanpypreprocess_job_spec.set_parameter("genes.min.cells", "")
scanpypreprocess_job_spec.set_parameter("genes.max.cells", "")
scanpypreprocess_job_spec.set_parameter("normalize", "1")
scanpypreprocess_job_spec.set_parameter("n.high.variance.genes", "3000")
scanpypreprocess_job_spec.set_parameter("compute.umap", "1")
scanpypreprocess_job_spec.set_parameter("compute.tsne", "1")
scanpypreprocess_job_spec.set_parameter("genome", "")
scanpypreprocess_job_spec.set_parameter("job.memory", "64Gb")
scanpypreprocess_job_spec.set_parameter("job.queue", "gpbeta-default")
scanpypreprocess_job_spec.set_parameter("job.cpuCount", "4")
scanpypreprocess_job_spec.set_parameter("job.walltime", "04:00:00")
genepattern.display(scanpypreprocess_task)

job99247 = gp.GPJob(genepattern.session.get(0), 99247)
genepattern.display(job99247)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00382')

GPJobWidget(job_number=99247)

As you can see there are two files returned after subsetting the highly variable genes. One is the full data set with the highly variable genes labeled, and the other is just the highly variable subset. When running multiple sections, this is the file that gets passed downstream. So, "donor_5_dim_reduce.h5ad" will be a small file with just the highly variable genes and the UMAP and TSNE coordinates already computed. This is the file we want to work with going forward and we will offically consider the pre-processing phase completed.

<div class="alert alert-info">
<b style="font-size:120%;">DownloadJobResultFile</b><br>
<b>Job File Url</b> select "donor_5_dim_reduce.h5ad" from the dropdown meny<br>
<b>File Name</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
</div>

In [46]:
genepattern.GPUIBuilder(DownloadJobResultFile,
                        description=DownloadJobResultFile_description,
                        parameters=DownloadJobResultFile_parameters)

UIBuilder(description='Download file from a GenePattern module job result.', function_import='DownloadJobResul…

# Visualization
<a href=#Table-of-Contents>back to top</a>  

In this section we will be taking a preliminary look at the data now that we have the UMAP and T-SNE coordinates of the data. We can clearly see that there is separation between the different 10x channels and that some cell types are separated. However, these methods are primarily for visualization and more analytical methods are needed before we can make any claims. In the next section we explore a matrix factorization algorithm, **CoGAPS**, which helps identify the underlying biological processes which drive the signal in the data.

<div class="alert alert-info">
<b style="font-size:120%;">TsnePlot</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Variable</b>channl<br>
<b>Variable Subset</b> --leave blank--<br>
</div>

In [47]:
genepattern.GPUIBuilder(TsnePlot,
                    description=TsnePlot_description,
                    parameters=TsnePlot_parameters)

UIBuilder(description='Create a t-sne plot of an AnnData object.', function_import='TsnePlot', name='TsnePlot'…

<div class="alert alert-info">
<b style="font-size:120%;">TsnePlot</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Variable</b> channel<br>
<b>Variable Subset</b> 1,2,3,4<br>
</div>

In [48]:
genepattern.GPUIBuilder(TsnePlot,
                    description=TsnePlot_description,
                    parameters=TsnePlot_parameters)

UIBuilder(description='Create a t-sne plot of an AnnData object.', function_import='TsnePlot', name='TsnePlot'…

<div class="alert alert-info">
<b style="font-size:120%;">TsnePlot</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Variable</b> channel<br>
<b>Variable Subset</b> 5,6,7,8<br>
</div>

In [49]:
genepattern.GPUIBuilder(TsnePlot,
                    description=TsnePlot_description,
                    parameters=TsnePlot_parameters)

UIBuilder(description='Create a t-sne plot of an AnnData object.', function_import='TsnePlot', name='TsnePlot'…

We can also see how well UMAP and T-SNE visualizations can separate cell type.

<div class="alert alert-info">
<b style="font-size:120%;">TsnePlot</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Variable</b> cell_type<br>
<b>Variable Subset</b> B cells, CD4 T cells, CD8 T cells, DCs, NK cells, Neutrophils, T cells, monocytes/macrophages, pDCs<br>
</div>

In [50]:
genepattern.GPUIBuilder(TsnePlot,
                    description=TsnePlot_description,
                    parameters=TsnePlot_parameters)

UIBuilder(description='Create a t-sne plot of an AnnData object.', function_import='TsnePlot', name='TsnePlot'…

<div class="alert alert-info">
<b style="font-size:120%;">UmapPlot</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Variable</b> cell_type<br>
<b>Variable Subset</b> B cells, CD4 T cells, CD8 T cells, DCs, NK cells, Neutrophils, T cells, monocytes/macrophages, pDCs<br>
</div>

In [51]:
genepattern.GPUIBuilder(UmapPlot,
                    description=UmapPlot_description,
                    parameters=UmapPlot_parameters)

UIBuilder(description='Create a UMAP plot of an AnnData object.', function_import='UmapPlot', name='UmapPlot',…

# CoGAPS Analysis
<a href=#Table-of-Contents>back to top</a>  

In this section we go through an entire **CoGAPS** workflow and carefully detail all of the considerations that need to be taken into account when running the algorithm. **CoGAPS** is a non-negative matrix factorization algorithm, which decomposes the original data into two lower-dimensional matrices. Matrix factorization methods in general rely on the principle that interrelated gene regulatory mechanisms, gene-gene interactions, and cellular interactions induce a low-dimensional structure within the fundamentally high-dimensional data produced with high-throughput technologies. Matrix Factorization techniques can learn this low-dimensional representation and have been successfully used in the past [7]. **CoGAPS** is similar to other non-negative matrix factorization algorithms, but it adds an additional sparsity constraint on the learned low-dimensional representation.

## Setting up a CoGAPS Run
<a href=#Table-of-Contents>back to top</a>  

We first use the next two cells to get an idea of the size and sparsity of the data. These are important numbers, since later on when setting the parameters for **CoGAPS** we need to understand the type of data we have.

<div class="alert alert-info">
<b style="font-size:120%;">GetDataSize</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
</div>

In [52]:
genepattern.GPUIBuilder(GetDataSize,
                    description=GetDataSize_description,
                    parameters=GetDataSize_parameters)

UIBuilder(description='Display number of features/samples in a data set.', function_import='GetDataSize', name…

<div class="alert alert-info">
<b style="font-size:120%;">GetDataSparsity</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
</div>

In [53]:
genepattern.GPUIBuilder(GetDataSparsity,
                    description=GetDataSparsity_description,
                    parameters=GetDataSparsity_parameters)  

UIBuilder(description='Display percentage of data that is zero.', function_import='GetDataSparsity', name='Get…

These next two cells will extract the gene and sample names from the data and store them in variables. This allows us to pass those variables to **CoGAPS** so that our data points are labelled correctly. We use the "RegexVariableNames" cell here for convenience, and simply tell it to match the entire name for each gene and sample.

<div class="alert alert-info">
<b style="font-size:120%;">RegexVariableNames</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Pattern</b> .*<br>
<b>Which Axis</b> features<br>
<b>variable name for regex output</b> gene_names<br>
</div>

In [54]:
genepattern.GPUIBuilder(RegexVariableNames,
                        description=RegexVariableNames_description,
                        parameters=RegexVariableNames_parameters)

UIBuilder(description='Parse AnnData column using regex pattern.', function_import='RegexVariableNames', name=…

<div class="alert alert-info">
<b style="font-size:120%;">RegexVariableNames</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Pattern</b> .*<br>
<b>Which Axis</b> samples<br>
<b>variable name for regex output</b> sample_names<br>
</div>

In [55]:
genepattern.GPUIBuilder(RegexVariableNames,
                        description=RegexVariableNames_description,
                        parameters=RegexVariableNames_parameters)

UIBuilder(description='Parse AnnData column using regex pattern.', function_import='RegexVariableNames', name=…

There are two things we must do before starting a **CoGAPS** run. First we must create a Cogapps Parameters file and second we must convert our data to a format that **CoGAPS** accepts. In the case of single-cell data this will almost always be an `.mtx` file due to the sparsity of the data. We can use the next cell to create the parameters file. There are a handful of parameters to consider so we describe their relevance here.

**Number of Patterns** - this is the most fundamental parameter. It controls the size of the low-dimensional representation of the data. The optimal value is not known and it is even debatable if there is a single best value for a given data set. It is likely that for different numbers of patterns , the underlying biological behavior will be learned at a different resolution. For this reason, it is common to run **CoGAPS** over a range of numbers of patterns and analyze the differences between the runs. In this case, we will start off with 20 patterns and see how the analysis goes. In future versions of the notebook we will explore a range of values for this parameter.

**Number of Iterations** - this controls how many iterations the algorithm will run for. Determining when the algorithm has converged is not an exact science, so typically a large value in neighborhood of 50,000 iterations is recommended to be safe.

**Random Seed** - this is the random seed, used for reproducibility

**Single Cell Data** - this should be set to TRUE when running on single-cell data. It changes the underlying mathematics of the algorithm to account for the sparsity present in single-cell data.

**Enable Sparse Optimization** - this has no effect on the mathematics of the algorithm and is purely used for improving the run time. It is recommended to set this to TRUE when the sparsity of the data is over 80% (i.e. 80% of the data is zero).

**Feature Names** and **Sample Names** - this is an optional field for setting the gene and sample names of the data. Since an `.mtx` file contains no text data, it is required in our case. Some other file formats make it possible to infer these names.

**Distributed Method** - this is perhaps the second most critical parameter to account for. By default, **CoGAPS** will run the factorization on the entire data set, using only a single cpu core. For large data sets, this makes it completely impractical to use since the run time is so long. When running on large data sets such as this one, this parameter enables a distributed version of the algorithm. This version of the algorithm splits the data into subsets and runs the standard algorithm on each subset in parallel. If the value of this parameter is "genome-wide" the data set is broken into subsets of genes, if the value is "single-cell" the data set is broken into subsets of cells. Once the parallel runs finish, there is an internal method for matching the common patterns between subsets and stitching them together to form a final result. This provides an additional benefit since patterns which are not robust across subsets are thrown out. In this way, the value for **Number of Patterns** is less critical since only robust patterns are kept [5].

**Number of Sets** - this parameter controls how many subsets are run in parallel when using the distributed version of the algorithm. In this case we break the data into 12 sets which gives us about 3000 cells per subset. We want to make sure each subset has at least 1000 or 2000 cells in order give robust results, but ideally we would want as many cells per set as possible.

<div class="alert alert-info">
<b style="font-size:120%;">CreateCogapsParameters</b><br>
<b>Output File Name</b> ica_cord_blood/donor_5_cogaps_params<br>
<b>Number of Patterns</b> 20<br>
<b>Number of Iterations</b> 50000<br>
<b>Random Seed</b> 42<br>
<b>Single Cell Data</b> TRUE<br>
<b>Enable Sparse Optimization</b> TRUE<br>
<b>Feature Names</b> gene_names<br>
<b>Sample Names</b> sample_names<br>
<b>Distributed Method</b> single-cell<br>
<b>Number of Sets</b> 12<br>
</div>

In [56]:
genepattern.GPUIBuilder(CreateCogapsParameters,
                        description=CreateCogapsParameters_description,
                        parameters=CreateCogapsParameters_parameters)

UIBuilder(description='Create a CoGAPS parameters object and save it to a file.', function_import='CreateCogap…

Now that we have our parameters file, we just have to make sure our data is in the correct format and then we can upload both files to the GenePattern server. This next cell converts the data to an ".mtx" file, and the following two cells handle the uploading.

<div class="alert alert-info">
<b style="font-size:120%;">ConvertToMtx</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Mtx File</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
</div>

In [57]:
genepattern.GPUIBuilder(ConvertToMtx,
                description=ConvertToMtx_description,
                parameters=ConvertToMtx_parameters)  

UIBuilder(description='Convert AnnData file to mtx.', function_import='ConvertToMtx', name='ConvertToMtx', par…

<div class="alert alert-info">
<b style="font-size:120%;">UploadFileToGenePatternServer</b><br>
<b>Local File</b> ica_cord_blood/donor_5_cogaps_params.rds<br>
<b>Remote File Name</b> ica_cord_blood_donor_5_cogaps_params.rds<br>
<b>varible name for url</b> param_url<br>
</div>

In [58]:
genepattern.GPUIBuilder(UploadFileToGenePatternServer,
                        description=UploadFileToGenePatternServer_description,
                        parameters=UploadFileToGenePatternServer_parameters)

UIBuilder(description='Upload the local file onto the GenePattern server.', function_import='UploadFileToGeneP…

<div class="alert alert-info">
<b style="font-size:120%;">UploadFileToGenePatternServer</b><br>
<b>Local File</b> ica_cord_blood/donor_5_preprocessed.mtx<br>
<b>Remote File Name</b> ica_cord_blood_donor_5_preprocessed.mtx<br>
<b>varible name for url</b> data_url<br>
</div>

In [59]:
genepattern.GPUIBuilder(UploadFileToGenePatternServer,
                        description=UploadFileToGenePatternServer_description,
                        parameters=UploadFileToGenePatternServer_parameters)

UIBuilder(description='Upload the local file onto the GenePattern server.', function_import='UploadFileToGeneP…

Finally we're ready to launch a **CoGAPS** job using the GenePattern module below. Notice that this module requires setting the **num patterns** and **num iterations** parameters directly in the module. These values will overwrite the parameter file so it's important to set these correctly. There are also some considerations we have to make in the "Job Options" section. We will want to set the **job queue** to "CoGAPS" and the **job cpuCount** to "12" since this is how many sets we are using.

<div class="alert alert-info">
<h3 style="margin-top: 0;"> CoGAPS Module Instructions<i class="fa fa-info-circle"></i></h3>
<ol>
    <li>Under "Basic Parameters" set <b>data file</b> to "{{ data_url }}"</li>
    <li>Under "Basic Parameters" set <b>output file</b> to "donor-5-patterns-20-cogaps-result"</li>
    <li>Under "Basic Parameters" set <b>num patterns</b> to "20"</li>
    <li>Under "Basic Parameters" set <b>num iterations</b> to "50000"</li>
    <li>Under "Parameter File" set <b>param file</b> to "{{ param_url }}"</li>
    <li>Under "Miscellaneous" set <b>transpose data</b> to "False"</li>
    <li>In the top right corner click the settings icon and click "Toggle Job Options" to enable this section</li>
    <li>Under "Job Options" set <b>job cpuCount</b> to "12"</li>
    <li>Under "Job Options" set <b>job queue</b> to "CoGAPS"</li>
</ol>
Note: do not include the quotes when setting parameters
</div>

<div class="alert alert-warning">
<h3 style="margin-top: 0;"> CoGAPS Running Time<i class="fa fa-info-circle"></i></h3>
This module launches a CoGAPS run that takes about 30 hours - it is recommended to use the pre-generated result if you are trying out the notebook.
</div>

In [7]:
cogaps_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00376')
cogaps_job_spec = cogaps_task.make_job_spec()
cogaps_job_spec.set_parameter("data.file", "{{ data_url }}")
cogaps_job_spec.set_parameter("output.file", "donor-5-patterns-20-cogaps-result")
cogaps_job_spec.set_parameter("num.patterns", "20")
cogaps_job_spec.set_parameter("num.iterations", "1000")
cogaps_job_spec.set_parameter("param.file", "{{ param_url }}")
cogaps_job_spec.set_parameter("transpose.data", "FALSE")
cogaps_job_spec.set_parameter("github.tag", "")
cogaps_job_spec.set_parameter("job.memory", "32 Gb")
cogaps_job_spec.set_parameter("job.queue", "CoGAPS")
cogaps_job_spec.set_parameter("job.cpuCount", "12")
cogaps_job_spec.set_parameter("job.walltime", "")
genepattern.display(cogaps_task)

job116783 = gp.GPJob(genepattern.session.get(0), 116783)
genepattern.display(job116783)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00376')

GPJobWidget(job_number=116783)

<div class="alert alert-info">
<b style="font-size:120%;">DownloadJobResultFile</b><br>
<b>Job File Url</b> select "donor-5-patterns-20-cogaps-result.rds" from the dropdown meny<br>
<b>File Name</b> ica_cord_blood/donor_5_patterns_20_cogaps_result.rds<br>
</div>

In [60]:
genepattern.GPUIBuilder(DownloadJobResultFile,
                        description=DownloadJobResultFile_description,
                        parameters=DownloadJobResultFile_parameters)

UIBuilder(description='Download file from a GenePattern module job result.', function_import='DownloadJobResul…

## Processing the CoGAPS Result
<a href=#Table-of-Contents>back to top</a>  

Now that we have the results from the **CoGAPS** run, we will want to merge it into our original **AnnData** file. This will allow us to overlay the results on the data which can make some nice visualizations.

<div class="alert alert-info">
<b style="font-size:120%;">ViewCogapsResult</b><br>
<b>Result File</b> ica_cord_blood/donor_5_patterns_20_cogaps_result.rds<br>
</div>

In [61]:
genepattern.GPUIBuilder(ViewCogapsResult,
                    description=ViewCogapsResult_description,
                    parameters=ViewCogapsResult_parameters)

UIBuilder(description='View the properties of a Cogaps result file.', function_import='ViewCogapsResult', name…

<div class="alert alert-info">
<b style="font-size:120%;">MergeCogapsResultWithAnnData</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Cogaps Result File</b> ica_cord_blood/donor_5_patterns_20_cogaps_result.rds<br>
<b>Output File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
</div>

In [62]:
genepattern.GPUIBuilder(MergeCogapsResultWithAnnData,
                description=MergeCogapsResultWithAnnData_description,
                parameters=MergeCogapsResultWithAnnData_parameters)

UIBuilder(description='Adds the patterns from the Cogaps result to the AnnData file.', function_import='MergeC…

## Visualizing Patterns
<a href=#Table-of-Contents>back to top</a>  

Visualizing the patterns learned from **CoGAPS** is an important task, but it is not enough to make definitive statements about the biological relevance of specific patterns - we will get to that in the next section. For now, we show some ways that patterns can visualized and point out some interesting examples. We note that this visualization can be applied to the results from all sample weight matrices learned from matrix factorization approaches [7].

<div class="alert alert-info">
<b style="font-size:120%;">PlotCogapsPatterns</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
<b>Visualization Coordinates</b> T-SNE<br>
<b>Patterns</b> --leave blank--<br>
<b>Phenotype</b> --leave blank--<br>
<b>Phenotype Values</b> --leave blank--<br>
<b>Subset</b> FALSE<br>
<b>Width</b> 4<br>
    
This will make a T-SNE plot of all CoGAPS patterns, with 4 plots per row.
</div>

In [63]:
genepattern.GPUIBuilder(PlotCogapsPatterns,
                        description=PlotCogapsPatterns_description,
                        parameters=PlotCogapsPatterns_parameters)

UIBuilder(description='Plots patterns overlaid on either a UMAP or TSNE plot of the original data.', function_…

The next two plots are interesting examples of patterns that seem to be strongly associated with a particular phenotype. In the next section we will make this association more precise, but it is worth looking at visualization like this to identify candidate patterns for later analysis.

<div class="alert alert-info">
<b style="font-size:120%;">PlotCogapsPatterns</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
<b>Visualization Coordinates</b> T-SNE<br>
<b>Patterns</b> 10<br>
<b>Phenotype</b> channel<br>
<b>Phenotype Values</b> --leave blank--<br>
<b>Subset</b> FALSE<br>
<b>Width</b> 2<br>
    
This will make a side by side T-SNE plot colored by pattern 10 and the channel each sample comes from.
</div>

In [64]:
genepattern.GPUIBuilder(PlotCogapsPatterns,
                        description=PlotCogapsPatterns_description,
                        parameters=PlotCogapsPatterns_parameters)

UIBuilder(description='Plots patterns overlaid on either a UMAP or TSNE plot of the original data.', function_…

<div class="alert alert-info">
<b style="font-size:120%;">PlotCogapsPatterns</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
<b>Visualization Coordinates</b> UMAP<br>
<b>Patterns</b> 4<br>
<b>Phenotype</b> cell_type<br>
<b>Phenotype Values</b> NK cells<br>
<b>Subset</b> FALSE<br>
<b>Width</b> 2<br>
    
This will make a side by side UMAP plot colored by pattern 4 and NK cells.
</div>

In [65]:
genepattern.GPUIBuilder(PlotCogapsPatterns,
                        description=PlotCogapsPatterns_description,
                        parameters=PlotCogapsPatterns_parameters)

UIBuilder(description='Plots patterns overlaid on either a UMAP or TSNE plot of the original data.', function_…

## Analyzing Patterns
<a href=#Table-of-Contents>back to top</a>  

In this section we dive into a more rigorous analysis of the patterns learned from **CoGAPS**. The types of analysis we might do breaks into several groups. First, when we have known phenotypes (e.g. donor, channel) we can directly see which patterns show a statistically significant separation of the phenotype. Second, we can do cell type identification using the marker genes we've previously described. By looking at which patterns are uniquely associated with the marker genes, we can label patterns as being an indicator for a given cell type. Finally, there are some patterns that aren't clearly associated with either of the former categories and so we must take a more exploratory approach to identifying their relevance.

### Known Phenotypes
<a href=#Table-of-Contents>back to top</a>  

In this case, we are trying to see if any patterns separate the different 10x channels. The null hypothesis would be that all 8 populations from the 8 different channels have the same distribution. We can test whether or not this is true with an ANOVA test. In this case, we use the non-parametric version of ANOVA, the Kruskal-Wallis test. We calculate the statistic and p-value for each of the patterns. Note that pattern 10, which was visually identified in the previous section, has a test statistic an order of magnitude larger than any other pattern. It seems clear that this pattern is separating the 10x channel, however the p-values for almost all patterns are "significant". Obviously, some correction needs to be used when calculating p-values in this manner and we will explore that in future versions of the notebook. For now, we justify the relevance of pattern 10 with the test statistic alone.

<div class="alert alert-info">
<b style="font-size:120%;">KruskalWallisPatternTest</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
<b>Phenotype</b> channel<br>
</div>

In [66]:
genepattern.GPUIBuilder(KruskalWallisPatternTest,
                        description=KruskalWallisPatternTest_description,
                        parameters=KruskalWallisPatternTest_parameters)

UIBuilder(description='Runs a Kruskal-Wallis test for a categorical phenotype across all patterns.', function_…

Now that we've identified pattern 10 as separating the 10x channels, we can use a post-hoc test to see specifically which channels are separated. The [Dunn Test](https://cran.r-project.org/web/packages/dunn.test/dunn.test.pdf) is a non-parametric test designed to work in conjunction with the Kruskal-Wallis test. We can see that channels 1,2,3,4 are strongly separated from channels 5,6,7,8 - but within those groups there is not as much separation. This is something we had already seen visually in the UMAP/TSNE plots.

<div class="alert alert-info">
<b style="font-size:120%;">DunnTest</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
<b>Pattern</b> 10<br>
<b>Phenotype</b> channel<br>
</div>

In [67]:

genepattern.GPUIBuilder(DunnTest,
                        description=DunnTest_description,
                        parameters=DunnTest_parameters)

UIBuilder(description='Runs a Dunn test for a categorical phenotype on a specific pattern.', function_import='…

### Cell Type Identification with CoGAPS
<a href=#Table-of-Contents>back to top</a>  

Here we do cell type identification by using the previously described marker genes. We can first take a look at the correlation between the pattern weights and the cell types found by **garnett**. This isn't necessarily informative since **garnett** cell types do not form a ground truth, but the comparison is still interesting. Only one pattern, 4, really seems associated with any of the **garnett** cell types (NK cells).

<div class="alert alert-info">
<b style="font-size:120%;">PlotCellTypePatternCorrelation</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
</div>

In [68]:
genepattern.GPUIBuilder(PlotCellTypePatternCorrelation,
    description=PlotCellTypePatternCorrelation_description,
    parameters=PlotCellTypePatternCorrelation_parameters)

UIBuilder(description='Plot a heatmap of corrlation between cell types and patterns.', function_import='PlotCe…

Here we demonstrate the cell type identification process using marker genes. We print out the top 3 patterns by rank for each marker. Using this we can see the patterns which are most influenced by all marker genes for a particular type.

<div class="alert alert-info">
<b style="font-size:120%;">CogapsCellTypeIdentification</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
<b>Marker Genes File</b> ica_cord_blood/markers.txt<br>
</div>

In [69]:
genepattern.GPUIBuilder(CogapsCellTypeIdentification,
                        description=CogapsCellTypeIdentification_description,
                        parameters=CogapsCellTypeIdentification_parameters)        

UIBuilder(description='Run cell type identification with Cogaps patterns.', function_import='CogapsCellTypeIde…

If we look at the marker genes for `NK cells` we can see that both marker genes have a high rank in pattern 4. This is consistent with what we saw visually and the correlation heatmap we made in the previous section. If we make a umap plot with just the `NK cells` from **garnett** highlighted, we can see a strong correlation with pattern 4. What's interesting is that the marker genes for `Neutrophils` also have a high rank in pattern 4, as well as pattern 11. However, if we plot those patterns on the `Neutrophils` generated by **garnett**, we can see there is little agreement. As is often the case, we have to do a deeper dive in order to get a sense of how cell types are related to patterns. This is done in the next section.

<div class="alert alert-info">
<b style="font-size:120%;">PlotCogapsPatterns</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
<b>Visualization Coordinates</b> UMAP<br>
<b>Patterns</b> 4, 14<br>
<b>Phenotype</b> cell_type<br>
<b>Phenotype Values</b> NK cells, Neutrophils<br>
<b>Subset</b> FALSE<br>
<b>Width</b> 2<br>
    
This will make a side by side UMAP plot colored by pattern 4 and 14 as well NK cells and Neutrophils.
</div>

In [70]:
genepattern.GPUIBuilder(PlotCogapsPatterns,
                        description=PlotCogapsPatterns_description,
                        parameters=PlotCogapsPatterns_parameters)

UIBuilder(description='Plots patterns overlaid on either a UMAP or TSNE plot of the original data.', function_…

### Exploring the Patterns
<a href=#Table-of-Contents>back to top</a>  

In most cases, the **CoGAPS** patterns will not so clearly identify groups such as the `NK cells` or the 10x channels. Part of a **CoGAPS** workflow is exploring the pattern space in order to determine the structure, or lack thereof, that it has learned from the original data. In this case, one interesting place to start would be pattern 14. It clearly is separating out a signal that was significant to the UMAP and TSNE algorithms, but that group of cells is not associated with any cell type or experimental condition. It is possible that it represents a phenotype that we haven't annotated yet, such a cell cycle. It could also be that **garnett** was inaccurate in labeling cell types or our marker file was incomplete. In either case, it makes sense to examine this more closely.

<div class="alert alert-info">
<b style="font-size:120%;">PlotCogapsPatterns</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
<b>Visualization Coordinates</b> UMAP<br>
<b>Patterns</b> 14<br>
<b>Phenotype</b> cell_type<br>
<b>Phenotype Values</b> --leave blank--<br>
<b>Subset</b> FALSE<br>
<b>Width</b> 2<br>
    
This will make a UMAP colored by pattern 14 as well as cell type.
</div>

In [71]:
genepattern.GPUIBuilder(PlotCogapsPatterns,
                        description=PlotCogapsPatterns_description,
                        parameters=PlotCogapsPatterns_parameters)

UIBuilder(description='Plots patterns overlaid on either a UMAP or TSNE plot of the original data.', function_…

Clearly there is not agreement between any single **garnett** cell type and pattern 14. However, if we look at the pattern ranks for our marker genes we can see that pattern 14 is strongly associated with the marker genes for B cells. Here we plot patterns 2 an 14, which have the highest marker ranks for B cells. It seems that **garnett** has some agreement in how to label `B cells`, but pattern 14 also has high weight in cells labeled `DCs` and `Neutrophils`. It is worth pointing out that these 3 cell types are the only ones to have any weight in pattern 14.

<div class="alert alert-info">
<b style="font-size:120%;">PlotCogapsPatterns</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
<b>Visualization Coordinates</b> UMAP<br>
<b>Patterns</b> 2, 14<br>
<b>Phenotype</b> cell_type<br>
<b>Phenotype Values</b> B cells, DCs, Neutrophils<br>
<b>Subset</b> TRUE<br>
<b>Width</b> 3<br>
    
This will make a side by side UMAP plot colored by pattern 2 and 14 as well as B cells, DCs, and Neutrophils - only the samples belonging to those cell types will be shown.
</div>

In [72]:
genepattern.GPUIBuilder(PlotCogapsPatterns,
                        description=PlotCogapsPatterns_description,
                        parameters=PlotCogapsPatterns_parameters)

UIBuilder(description='Plots patterns overlaid on either a UMAP or TSNE plot of the original data.', function_…

<div class="alert alert-info">
<b style="font-size:120%;">PlotCogapsPatterns</b><br>
<b>AnnData File</b> ica_cord_blood/donor_5_cogaps_result.h5ad<br>
<b>Visualization Coordinates</b> UMAP<br>
<b>Patterns</b> 2, 14<br>
<b>Phenotype</b> cell_type<br>
<b>Phenotype Values</b> CD4 T cells, CD8 T cells, T cells, NK cells, monocytes/macrophages<br>
<b>Subset</b> TRUE<br>
<b>Width</b> 3<br>
    
This will make a side by side UMAP plot colored by pattern 2 and 14 as well as CD4 T cells, CD8 T cells, T cells, NK cells, monocytes/macrophages - only the samples belonging to those cell types will be shown.
</div>

In the next plot we show that all other cell types have no weight in pattern 14, however pattern 2 still has weight in seemingly all cell types. This suggests pattern 2 is a larger effect not specific to a cell type, but that pattern 14 is pulling out something specific to `B cells`, `DCs`, and `Neutrophils`.

In [73]:
genepattern.GPUIBuilder(PlotCogapsPatterns,
                        description=PlotCogapsPatterns_description,
                        parameters=PlotCogapsPatterns_parameters)

UIBuilder(description='Plots patterns overlaid on either a UMAP or TSNE plot of the original data.', function_…

The next three plots split the cell types up so there is no overlap in the UMAP. This allows us to see exactly which cell types UMAP can separate, and gives us some easy targets for finding **CoGAPS** patterns for. In future versions of the notebook we will do a complete accounting for all the patterns learned in **CoGAPS**. It will be interesting to see how well **CoGAPS** can separate the subtypes of T cells, something that is clearly not happening in the UMAP. We will also want to see if **CoGAPS** can separate the other cell types that overlap as well as any `Unknown` cell types, which make up roughly 1/3 of the total samples.

<div class="alert alert-info">
<b style="font-size:120%;">UmapPlot</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Variable</b> cell_type<br>
<b>Variable Subset</b> B cells, monocytes/macrophages<br>
</div>

In [74]:
genepattern.GPUIBuilder(UmapPlot,
                        description=UmapPlot_description,
                        parameters=UmapPlot_parameters)

UIBuilder(description='Create a UMAP plot of an AnnData object.', function_import='UmapPlot', name='UmapPlot',…

<div class="alert alert-info">
<b style="font-size:120%;">UmapPlot</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Variable</b> cell_type<br>
<b>Variable Subset</b> CD4 T cells, CD8 T cells, T cells, pDCs Dcs<br>
</div>

In [75]:
genepattern.GPUIBuilder(UmapPlot,
                        description=UmapPlot_description,
                        parameters=UmapPlot_parameters)

UIBuilder(description='Create a UMAP plot of an AnnData object.', function_import='UmapPlot', name='UmapPlot',…

<div class="alert alert-info">
<b style="font-size:120%;">UmapPlot</b><br>
<b>AnnData File Url</b> ica_cord_blood/donor_5_preprocessed.h5ad<br>
<b>Variable</b> cell_type<br>
<b>Variable Subset</b> NK cells, Neutrophils<br>
</div>

In [76]:
genepattern.GPUIBuilder(UmapPlot,
                        description=UmapPlot_description,
                        parameters=UmapPlot_parameters)

UIBuilder(description='Create a UMAP plot of an AnnData object.', function_import='UmapPlot', name='UmapPlot',…

# References
<a href=#Table-of-Contents>back to top</a>  

1. Census of Immune Cells - Contributors: Bo Li, Monika S Kowalczyk, Danielle Dionne, Orr Ashenberg, Marcin Tabaka, Timothy Tickle, Jane Lee, Karthik Shekhar, Michal Slyper, Julia Waldman, Orit Rozenblatt-Rosen, Aviv Regev. Collaborating organisations: Broad Institute, Massachusetts Institute of Technology, Howard Hughes Medical Institute.

2. Fertig, E.J., et al. (2010). CoGAPS: an R/C++ package to identify patterns and biological process activity in transcriptomic data. Bioinformatics 26: 2792-2793.

3. Lun, A.T.L., et al. (2016). A step-by-step workflow for low-level analysis of single-cell RNA-seq data with Bioconductor. F1000Res., 5, 2122.

4. Pliner, H.A., et al. (2019). Supervised classification enables rapid annotation of cell atlases. bioRxiv

5. Stein-O'Brien, G.L. et al. (2017). PatternMarkers &amp; GWCoGAPS for novel data-driven biomarkers via whole transcriptome NMF. Bioinformatics 33(12): 1892-1894.

6. Stein-O'Brien, G.L. et al. (2018). Decomposing cell identity for transfer learning across cellular measurements, platforms, tissues, and species. BioRxiv

7. Stein-O'Brien, G.L. et al. (2018). Enter the Matrix: Factorization Uncovers Knowledge from Omics. Trends in Genetics, 34(10).

8. Wolf, A.F., et al. (2018). SCANPY: large-scale single-cell gene expression data analysis. Genome Biology. 19:15